# Truck Classification Counts

In [1]:
#import relevant packages
import pandas as pd, numpy as np, os, warnings, datetime
import matplotlib.pyplot as plt, seaborn as sns

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
#specify data/output path
data_path = 'Z:/PL43_TFR/Cleaned Data/'
output_path = 'Z:/PL43_TFR/Cleaned Data/Output/'

## Stations

In [3]:
#county names to identify station counties
counties = pd.read_csv(data_path + 'counties.csv')
counties.head(2)

,Number,County,District
0,1,Adair,8
1,2,Allen,3


In [4]:
#clean up stations
station_stats = pd.read_parquet(data_path + 'stations_stats_complete_clean_updated.parquet')
station_stats['FC_New'] = station_stats['FC_New'].astype(int)
station_stats['FC_Old'] = station_stats['FC_Old'].astype(int)
station_stats['Number'] = station_stats['Station'].str[:3].astype(int)
station_stats['Route'] = station_stats['RT_UNIQUE'].apply(lambda x: '-'.join(x.split('-')[1:3]))
station_stats.head(2)

,Station,Vol_Obs,Vol_Total,Vol_Complete,Vol_Clean,Class_Obs,Class_Total,Class_Complete,Class_Clean,RT_UNIQUE,MP,FC_New,FC_Old,Number,Route
0,001P34,3282.0,3652.0,89.87,89.70,11856.0,87648.0,13.53,13.53,001-KY-0080 -000,20.05,5,7,1,KY-0080
1,005P71,7096.0,7304.0,97.15,97.04,154716.0,175296.0,88.26,87.79,005-LN-9008 -000,10.10,2,12,5,LN-9008


In [5]:
station_stats = pd.merge(station_stats, counties, on='Number', how='left')
stations = station_stats[['Station','County', 'Route', 'MP', 'FC_Old', 'FC_New']]
stations.head()

,Station,County,Route,MP,FC_Old,FC_New
0,001P34,Adair,KY-0080,20.05,7,5
1,005P71,Barren,LN-9008,10.10,12,2
2,007P31,Bell,US-0025E,18.20,2,3
3,008P63,Boone,I -0075,176.00,11,1
4,009P26,Bourbon,US-0068,3.85,14,3


### Get number of times of each day of week occurs in each month in each calendar year

In [6]:
years = list(range(2017,2022))
months = list(range(1,13))
dow = list(range(0,7))

In [7]:
import calendar
from datetime import datetime
yr = []
month = []
wday = []
numdays = []
mrange = []
weekdays = []
weekends = []
for i in years:
    for j in months:
        wkdays = 0
        wkends = 0
        for k in dow:
            yr.append(i)
            month.append(j)
            mrange.append(calendar.monthrange(i, j)[1])
            wday.append(k)
            nd = len([1 for l in calendar.monthcalendar(i,j) if l[k] != 0])
            numdays.append(nd)
            if k < 5:
                wkdays += nd
            else:
                wkends += nd
        weekdays.extend([wkdays]*7)
        weekends.extend([wkends]*7)

In [8]:
# put the results of the analysis into a dtaframe
wjm = pd.DataFrame(data = {'Year':yr, 'Month':month, 'dow':wday, 'ndays_month':mrange, 
                           'nwkdays': weekdays, 'nwkends': weekends, 'W_jm':numdays})
wjm.head(8)

,Year,Month,dow,ndays_month,nwkdays,nwkends,W_jm
0,2017,1,0,31,22,9,5
1,2017,1,1,31,22,9,5
2,2017,1,2,31,22,9,4
3,2017,1,3,31,22,9,4
4,2017,1,4,31,22,9,4
5,2017,1,5,31,22,9,4
6,2017,1,6,31,22,9,5
7,2017,2,0,28,20,8,4


## Class Data Prep

In [9]:
class_data = pd.read_parquet(data_path + 'classification_5years_clean_all_cols.parquet')
class_data = class_data.iloc[:,3:24]

class_data['motorcycles'] = class_data['Class1']
class_data['passenger_cars'] = class_data['Class2']
class_data['other2axle4tire'] = class_data['Class3']

class_data['bus'] = class_data['Class4']
class_data['singleunit'] = class_data.iloc[:,12:15].sum(axis=1)
class_data['combination'] = class_data.iloc[:,15:21].sum(axis=1)
class_data['Truck_Count'] = class_data.iloc[:,12:21].sum(axis=1)  # Xu: Buses should not be included #addressed

class_data['naxle2'] = class_data[['Class1','Class2','Class3','Class4','Class5']].sum(axis=1)
class_data['naxle3'] = class_data['Class6']
class_data['naxle4'] = class_data[['Class7','Class8']].sum(axis=1)
class_data['naxle5'] = class_data[['Class9','Class11']].sum(axis=1)
class_data['naxle6'] = class_data[['Class10','Class12']].sum(axis=1)
class_data['naxle7'] = class_data['Class13']

#create date column from separate year, month, day columns
class_data['date'] = pd.to_datetime(class_data[['Year','Month','Day']])
#get day of week from datetime
class_data['dow'] = class_data['date'].dt.dayofweek
#create a new column to check if day of week is a weekend
class_data['weekend'] = class_data['dow'].map(lambda x: 1 if x>4 else 0)
class_data.head()

,Station,Dir,LaneID,Year,Month,Day,Hour,TotalCount,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,motorcycles,passenger_cars,other2axle4tire,bus,singleunit,combination,Truck_Count,naxle2,naxle3,naxle4,naxle5,naxle6,naxle7,date,dow,weekend
0,001P34,3,1,2017,1,1,0,24,0,24,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,24,0,0,0,0,0,2017-01-01,6,1
1,001P34,3,1,2017,1,1,1,15,0,14,1,0,0,0,0,0,0,0,0,0,0,0,14,1,0,0,0,0,15,0,0,0,0,0,2017-01-01,6,1
2,001P34,3,1,2017,1,1,2,6,0,6,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,6,0,0,0,0,0,2017-01-01,6,1
3,001P34,3,1,2017,1,1,3,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,2017-01-01,6,1
4,001P34,3,1,2017,1,1,4,12,0,12,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,12,0,0,0,0,0,2017-01-01,6,1


In [10]:
cols = ['motorcycles','passenger_cars','other2axle4tire','bus','singleunit','combination','Truck_Count','TotalCount']
class_data.info(show_counts=True)
display(class_data[cols].describe().transpose())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5041032 entries, 0 to 5041031
Data columns (total 37 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Station          5041032 non-null  object        
 1   Dir              5041032 non-null  int32         
 2   LaneID           5041032 non-null  int32         
 3   Year             5041032 non-null  int32         
 4   Month            5041032 non-null  int32         
 5   Day              5041032 non-null  int32         
 6   Hour             5041032 non-null  int32         
 7   TotalCount       5041032 non-null  int32         
 8   Class1           5041032 non-null  int32         
 9   Class2           5041032 non-null  int32         
 10  Class3           5041032 non-null  int32         
 11  Class4           5041032 non-null  int32         
 12  Class5           5041032 non-null  int32         
 13  Class6           5041032 non-null  int32         
 14  Cl

,count,mean,std,min,25%,50%,75%,max
motorcycles,5041032.0,0.588214,2.401827,0.0,0.0,0.0,1.0,340.0
passenger_cars,5041032.0,146.478785,212.676838,0.0,17.0,65.0,182.0,2920.0
other2axle4tire,5041032.0,39.793610,51.166178,0.0,5.0,21.0,55.0,726.0
bus,5041032.0,1.632805,3.874523,0.0,0.0,0.0,2.0,658.0
singleunit,5041032.0,8.460926,12.664244,0.0,1.0,4.0,11.0,393.0
combination,5041032.0,17.865162,43.014553,0.0,0.0,2.0,11.0,1127.0
Truck_Count,5041032.0,26.326088,51.436708,0.0,1.0,7.0,24.0,1142.0
TotalCount,5041032.0,214.819501,291.554549,0.0,27.0,102.0,277.0,2985.0


In [11]:
class_data[class_data.bus == 658]

,Station,Dir,LaneID,Year,Month,Day,Hour,TotalCount,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,motorcycles,passenger_cars,other2axle4tire,bus,singleunit,combination,Truck_Count,naxle2,naxle3,naxle4,naxle5,naxle6,naxle7,date,dow,weekend
1400008,051P58,5,1,2018,3,2,16,1033,0,245,9,658,93,0,0,18,10,0,0,0,0,0,245,9,658,93,28,121,1005,0,18,10,0,0,2018-03-02,4,0


In [12]:
stations[stations.Station == '051P58']

,Station,County,Route,MP,FC_Old,FC_New
35,051P58,Henderson,US-0041,18.25,14,3


In [13]:
# Xu: Buses max value 658 per lane per hour? Did you check which station and when? 

#it is on station 051P58 on US-41 in henderson county. March 2, 2018. 

In [14]:
# Xu: Add comments to below sections

### Data quality

In [15]:
def ideal_hours_covered(year):
    if year%4 ==0:
        ideal_hours = 8784
    else:
        ideal_hours = 8760
    return ideal_hours

In [16]:
#across lanes and direction
dt_intermediate = class_data.groupby(['Station', 'Year', 'Month', 'Day', 'Hour'], as_index=False).agg({'TotalCount':'count'})
dt_intermediate.head()

,Station,Year,Month,Day,Hour,TotalCount
0,001P34,2017,1,1,0,2
1,001P34,2017,1,1,1,2
2,001P34,2017,1,1,2,2
3,001P34,2017,1,1,3,2
4,001P34,2017,1,1,4,2


In [17]:
#get the percent data coverage for each station-year
data_quality = dt_intermediate.groupby(['Station', 'Year'], as_index=False).agg({'TotalCount':'count'})
data_quality['ideal_coverage'] = data_quality['Year'].map(ideal_hours_covered)
data_quality['year_percent_clean_data'] = data_quality['TotalCount']*100/data_quality['ideal_coverage']
data_quality.head()

,Station,Year,TotalCount,ideal_coverage,year_percent_clean_data
0,001P34,2017,5928,8760,67.671233
1,005P71,2017,4656,8760,53.150685
2,005P71,2018,8208,8760,93.698630
3,005P71,2019,8544,8760,97.534247
4,005P71,2020,8616,8784,98.087432


In [18]:
stations.head()

,Station,County,Route,MP,FC_Old,FC_New
0,001P34,Adair,KY-0080,20.05,7,5
1,005P71,Barren,LN-9008,10.10,12,2
2,007P31,Bell,US-0025E,18.20,2,3
3,008P63,Boone,I -0075,176.00,11,1
4,009P26,Bourbon,US-0068,3.85,14,3


In [19]:
#merge data quality information onto stations
stations = pd.merge(stations, data_quality[['Station','Year', 'year_percent_clean_data']], on=['Station'], how='left')
stations.head()

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data
0,001P34,Adair,KY-0080,20.05,7,5,2017.0,67.671233
1,005P71,Barren,LN-9008,10.10,12,2,2017.0,53.150685
2,005P71,Barren,LN-9008,10.10,12,2,2018.0,93.698630
3,005P71,Barren,LN-9008,10.10,12,2,2019.0,97.534247
4,005P71,Barren,LN-9008,10.10,12,2,2020.0,98.087432


### AADTT

In [20]:
unique_value = lambda x: x.unique()[0] 

In [21]:
%%time
#sum volume across lanes and direction.
truck_agg_hr = class_data.groupby(['Station','Year', 'Month', 'dow','Hour'], as_index=False).agg({
                                'TotalCount':'sum','naxle2':'sum','naxle3':'sum', 'naxle4':'sum',
                                'naxle5':'sum','naxle6':'sum','naxle7':'sum','motorcycles':'sum',
                                'passenger_cars':'sum','other2axle4tire':'sum','bus':'sum',
                                'singleunit':'sum','combination':'sum','Truck_Count':'sum',
                                'weekend':unique_value,'Day': 'nunique',})

#change columns names
truck_agg_hr.columns = ['Station', 'Year', 'Month', 'dow', 'Hour', 'TotalCount', 'naxle2','naxle3', 'naxle4', 
                        'naxle5', 'naxle6', 'naxle7', 'motorcycles','passenger_cars', 'other2axle4tire', 
                        'bus', 'singleunit', 'combination', 'Truck_Count', 'isweekend', 'ndayswithdata']

truck_agg_hr.head()

CPU times: total: 16 s
Wall time: 16.9 s


,Station,Year,Month,dow,Hour,TotalCount,naxle2,naxle3,naxle4,naxle5,naxle6,naxle7,motorcycles,passenger_cars,other2axle4tire,bus,singleunit,combination,Truck_Count,isweekend,ndayswithdata
0,001P34,2017,1,0,0,41,41,0,0,0,0,0,0,36,4,0,1,0,1,0,5
1,001P34,2017,1,0,1,26,26,0,0,0,0,0,0,25,0,0,1,0,1,0,5
2,001P34,2017,1,0,2,40,39,0,0,1,0,0,0,36,1,0,2,1,3,0,5
3,001P34,2017,1,0,3,106,106,0,0,0,0,0,1,91,13,0,1,0,1,0,5
4,001P34,2017,1,0,4,128,126,0,1,1,0,0,0,106,19,0,1,2,3,0,5


In [22]:
#average hourly truck volume for each day of week (dow) in a month typical month-dow-hour
truck_agg_hr['typicalhr_naxle2'] = truck_agg_hr['naxle2']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_naxle3'] = truck_agg_hr['naxle3']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_naxle4'] = truck_agg_hr['naxle4']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_naxle5'] = truck_agg_hr['naxle5']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_naxle6'] = truck_agg_hr['naxle6']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_naxle7'] = truck_agg_hr['naxle7']/truck_agg_hr['ndayswithdata']

truck_agg_hr['typicalhr_totalcount'] = truck_agg_hr['TotalCount']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_motorcycles'] = truck_agg_hr['motorcycles']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_passenger_cars'] = truck_agg_hr['passenger_cars']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_other2axle4tire'] = truck_agg_hr['other2axle4tire']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_bus'] = truck_agg_hr['bus']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_singleunit'] = truck_agg_hr['singleunit']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_combination'] = truck_agg_hr['combination']/truck_agg_hr['ndayswithdata']
truck_agg_hr['typicalhr_Truck_Count'] = truck_agg_hr['Truck_Count']/truck_agg_hr['ndayswithdata']
truck_agg_hr.head()

,Station,Year,Month,dow,Hour,TotalCount,naxle2,naxle3,naxle4,naxle5,naxle6,naxle7,motorcycles,passenger_cars,other2axle4tire,bus,singleunit,combination,Truck_Count,isweekend,ndayswithdata,typicalhr_naxle2,typicalhr_naxle3,typicalhr_naxle4,typicalhr_naxle5,typicalhr_naxle6,typicalhr_naxle7,typicalhr_totalcount,typicalhr_motorcycles,typicalhr_passenger_cars,typicalhr_other2axle4tire,typicalhr_bus,typicalhr_singleunit,typicalhr_combination,typicalhr_Truck_Count
0,001P34,2017,1,0,0,41,41,0,0,0,0,0,0,36,4,0,1,0,1,0,5,8.2,0.0,0.0,0.0,0.0,0.0,8.2,0.0,7.2,0.8,0.0,0.2,0.0,0.2
1,001P34,2017,1,0,1,26,26,0,0,0,0,0,0,25,0,0,1,0,1,0,5,5.2,0.0,0.0,0.0,0.0,0.0,5.2,0.0,5.0,0.0,0.0,0.2,0.0,0.2
2,001P34,2017,1,0,2,40,39,0,0,1,0,0,0,36,1,0,2,1,3,0,5,7.8,0.0,0.0,0.2,0.0,0.0,8.0,0.0,7.2,0.2,0.0,0.4,0.2,0.6
3,001P34,2017,1,0,3,106,106,0,0,0,0,0,1,91,13,0,1,0,1,0,5,21.2,0.0,0.0,0.0,0.0,0.0,21.2,0.2,18.2,2.6,0.0,0.2,0.0,0.2
4,001P34,2017,1,0,4,128,126,0,1,1,0,0,0,106,19,0,1,2,3,0,5,25.2,0.0,0.2,0.2,0.0,0.0,25.6,0.0,21.2,3.8,0.0,0.2,0.4,0.6


In [23]:
#typical dow day (sum across 24 hours for each dow)
truck_agg_dow = truck_agg_hr.groupby(['Station','Year', 'Month', 'dow'], as_index=False).agg({
                        'Hour':'count','typicalhr_totalcount':'sum','typicalhr_naxle2':'sum',
                        'typicalhr_naxle3':'sum','typicalhr_naxle4':'sum','typicalhr_naxle5':'sum',
                        'typicalhr_naxle6':'sum','typicalhr_naxle7':'sum','typicalhr_motorcycles':'sum',
                        'typicalhr_passenger_cars':'sum','typicalhr_other2axle4tire':'sum',
                        'typicalhr_bus':'sum','typicalhr_singleunit':'sum','typicalhr_combination':'sum',
                        'typicalhr_Truck_Count':'sum','isweekend':unique_value,})

truck_agg_dow.columns = ['Station', 'Year', 'Month', 'dow', 'Hour', 'typicaldow_totalcount',
                         'typicaldow_naxle2', 'typicaldow_naxle3', 'typicaldow_naxle4',
                         'typicaldow_naxle5', 'typicaldow_naxle6', 'typicaldow_naxle7',
                         'typicaldow_motorcycles', 'typicaldow_passenger_cars',
                         'typicaldow_other2axle4tire', 'typicaldow_bus', 'typicaldow_singleunit',
                         'typicaldow_combination', 'typicaldow_Truck_Count', 'isweekend']

truck_agg_dow.head()

,Station,Year,Month,dow,Hour,typicaldow_totalcount,typicaldow_naxle2,typicaldow_naxle3,typicaldow_naxle4,typicaldow_naxle5,typicaldow_naxle6,typicaldow_naxle7,typicaldow_motorcycles,typicaldow_passenger_cars,typicaldow_other2axle4tire,typicaldow_bus,typicaldow_singleunit,typicaldow_combination,typicaldow_Truck_Count,isweekend
0,001P34,2017,1,0,24,2522.40,2481.60,13.40,8.20,19.20,0.0,0.0,6.20,2080.20,320.60,9.20,79.00,27.20,106.20,0
1,001P34,2017,1,1,24,2789.80,2726.00,26.00,11.00,26.60,0.2,0.0,9.20,2279.60,342.20,14.00,108.40,36.40,144.80,0
2,001P34,2017,1,2,24,2891.25,2828.00,31.25,9.75,21.75,0.5,0.0,9.00,2377.00,350.00,15.25,109.50,30.50,140.00,0
3,001P34,2017,1,3,24,2671.25,2627.50,12.75,8.75,22.25,0.0,0.0,6.25,2236.25,298.75,13.00,86.75,30.25,117.00,0
4,001P34,2017,1,4,24,2851.50,2815.25,11.25,6.25,18.75,0.0,0.0,7.75,2399.75,314.25,14.50,90.50,24.75,115.25,0


In [24]:
truck_agg_dow = pd.merge(truck_agg_dow, wjm, on=['Year','Month','dow'], how='left')

truck_agg_dow['wjm_naxle2'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_naxle2']
truck_agg_dow['wjm_naxle3'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_naxle3']
truck_agg_dow['wjm_naxle4'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_naxle4']
truck_agg_dow['wjm_naxle5'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_naxle5']
truck_agg_dow['wjm_naxle6'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_naxle6']
truck_agg_dow['wjm_naxle7'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_naxle7']

truck_agg_dow['wjm_TotalCount'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_totalcount']
truck_agg_dow['wjm_motorcycles'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_motorcycles']
truck_agg_dow['wjm_passenger_cars'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_passenger_cars']
truck_agg_dow['wjm_other2axle4tire'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_other2axle4tire']
truck_agg_dow['wjm_bus'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_bus']
truck_agg_dow['wjm_singleunit'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_singleunit']
truck_agg_dow['wjm_combination'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_combination']
truck_agg_dow['wjm_Truck_Count'] = truck_agg_dow['W_jm'] * truck_agg_dow['typicaldow_Truck_Count']

truck_agg_dow.head()

,Station,Year,Month,dow,Hour,typicaldow_totalcount,typicaldow_naxle2,typicaldow_naxle3,typicaldow_naxle4,typicaldow_naxle5,typicaldow_naxle6,typicaldow_naxle7,typicaldow_motorcycles,typicaldow_passenger_cars,typicaldow_other2axle4tire,typicaldow_bus,typicaldow_singleunit,typicaldow_combination,typicaldow_Truck_Count,isweekend,ndays_month,nwkdays,nwkends,W_jm,wjm_naxle2,wjm_naxle3,wjm_naxle4,wjm_naxle5,wjm_naxle6,wjm_naxle7,wjm_TotalCount,wjm_motorcycles,wjm_passenger_cars,wjm_other2axle4tire,wjm_bus,wjm_singleunit,wjm_combination,wjm_Truck_Count
0,001P34,2017,1,0,24,2522.40,2481.60,13.40,8.20,19.20,0.0,0.0,6.20,2080.20,320.60,9.20,79.00,27.20,106.20,0,31,22,9,5,12408.0,67.0,41.0,96.0,0.0,0.0,12612.0,31.0,10401.0,1603.0,46.0,395.0,136.0,531.0
1,001P34,2017,1,1,24,2789.80,2726.00,26.00,11.00,26.60,0.2,0.0,9.20,2279.60,342.20,14.00,108.40,36.40,144.80,0,31,22,9,5,13630.0,130.0,55.0,133.0,1.0,0.0,13949.0,46.0,11398.0,1711.0,70.0,542.0,182.0,724.0
2,001P34,2017,1,2,24,2891.25,2828.00,31.25,9.75,21.75,0.5,0.0,9.00,2377.00,350.00,15.25,109.50,30.50,140.00,0,31,22,9,4,11312.0,125.0,39.0,87.0,2.0,0.0,11565.0,36.0,9508.0,1400.0,61.0,438.0,122.0,560.0
3,001P34,2017,1,3,24,2671.25,2627.50,12.75,8.75,22.25,0.0,0.0,6.25,2236.25,298.75,13.00,86.75,30.25,117.00,0,31,22,9,4,10510.0,51.0,35.0,89.0,0.0,0.0,10685.0,25.0,8945.0,1195.0,52.0,347.0,121.0,468.0
4,001P34,2017,1,4,24,2851.50,2815.25,11.25,6.25,18.75,0.0,0.0,7.75,2399.75,314.25,14.50,90.50,24.75,115.25,0,31,22,9,4,11261.0,45.0,25.0,75.0,0.0,0.0,11406.0,31.0,9599.0,1257.0,58.0,362.0,99.0,461.0


In [25]:
#sum across dows
truck_mn = truck_agg_dow.groupby(['Station','Year', 'Month'], as_index=False).agg({   
                                         'wjm_naxle2':'sum','wjm_naxle3':'sum','wjm_naxle4':'sum',
                                         'wjm_naxle5':'sum','wjm_naxle6':'sum','wjm_naxle7':'sum',
    
                                'wjm_TotalCount':'sum', 'wjm_motorcycles':'sum', 'wjm_passenger_cars':'sum',
                                'wjm_other2axle4tire':'sum', 'wjm_bus':'sum', 'wjm_singleunit':'sum', 
                                'wjm_combination':'sum', 'wjm_Truck_Count':'sum', 'ndays_month':unique_value,})



truck_mn.head()

,Station,Year,Month,wjm_naxle2,wjm_naxle3,wjm_naxle4,wjm_naxle5,wjm_naxle6,wjm_naxle7,wjm_TotalCount,wjm_motorcycles,wjm_passenger_cars,wjm_other2axle4tire,wjm_bus,wjm_singleunit,wjm_combination,wjm_Truck_Count,ndays_month
0,001P34,2017,1,77933.000000,443.000000,219.000000,521.0,3.0,0.0,79119.0,184.000000,65886.0,9583.0,318.0,2423.0,725.000000,3148.000000,31
1,001P34,2017,2,78357.000000,384.000000,283.000000,599.0,6.0,0.0,79629.0,119.000000,64668.0,11182.0,310.0,2489.0,861.000000,3350.000000,28
2,001P34,2017,3,87814.666667,454.666667,267.666667,683.0,1.0,0.0,89221.0,113.333333,71921.0,12972.0,393.0,2901.0,920.666667,3821.666667,31
3,001P34,2017,4,86985.000000,549.000000,256.000000,601.0,2.0,0.0,88393.0,170.000000,68095.0,15735.0,356.0,3213.0,824.000000,4037.000000,30
4,001P34,2017,5,89642.000000,625.000000,299.000000,631.0,3.0,0.0,91200.0,170.000000,69399.0,16774.0,381.0,3601.0,875.000000,4476.000000,31


In [26]:
truck_mn['MADT_naxle2'] = truck_mn['wjm_naxle2']/truck_mn['ndays_month']
truck_mn['MADT_naxle3'] = truck_mn['wjm_naxle3']/truck_mn['ndays_month']
truck_mn['MADT_naxle4'] = truck_mn['wjm_naxle4']/truck_mn['ndays_month']
truck_mn['MADT_naxle5'] = truck_mn['wjm_naxle5']/truck_mn['ndays_month']
truck_mn['MADT_naxle6'] = truck_mn['wjm_naxle6']/truck_mn['ndays_month']
truck_mn['MADT_naxle7'] = truck_mn['wjm_naxle7']/truck_mn['ndays_month']

truck_mn['MADT_TotalCount'] = truck_mn['wjm_TotalCount']/truck_mn['ndays_month']
truck_mn['MADT_motorcycles'] = truck_mn['wjm_motorcycles']/truck_mn['ndays_month']
truck_mn['MADT_passenger_cars'] = truck_mn['wjm_passenger_cars']/truck_mn['ndays_month']
truck_mn['MADT_other2axle4tire'] = truck_mn['wjm_other2axle4tire']/truck_mn['ndays_month']
truck_mn['MADT_bus'] = truck_mn['wjm_bus']/truck_mn['ndays_month']
truck_mn['MADT_singleunit'] = truck_mn['wjm_singleunit']/truck_mn['ndays_month']
truck_mn['MADT_combination'] = truck_mn['wjm_combination']/truck_mn['ndays_month']
truck_mn['MADT_Truck_Count'] = truck_mn['wjm_Truck_Count']/truck_mn['ndays_month']

truck_mn['dm_MADT_naxle2'] = truck_mn['ndays_month'] * truck_mn['MADT_naxle2']
truck_mn['dm_MADT_naxle3'] = truck_mn['ndays_month'] * truck_mn['MADT_naxle3']
truck_mn['dm_MADT_naxle4'] = truck_mn['ndays_month'] * truck_mn['MADT_naxle4']
truck_mn['dm_MADT_naxle5'] = truck_mn['ndays_month'] * truck_mn['MADT_naxle5']
truck_mn['dm_MADT_naxle6'] = truck_mn['ndays_month'] * truck_mn['MADT_naxle6']
truck_mn['dm_MADT_naxle7'] = truck_mn['ndays_month'] * truck_mn['MADT_naxle7']

truck_mn['dm_MADT_TotalCount'] = truck_mn['ndays_month'] * truck_mn['MADT_TotalCount']
truck_mn['dm_MADT_motorcycles'] = truck_mn['ndays_month'] * truck_mn['MADT_motorcycles']
truck_mn['dm_MADT_passenger_cars'] = truck_mn['ndays_month'] * truck_mn['MADT_passenger_cars']
truck_mn['dm_MADT_other2axle4tire'] = truck_mn['ndays_month'] * truck_mn['MADT_other2axle4tire']
truck_mn['dm_MADT_bus'] = truck_mn['ndays_month'] * truck_mn['MADT_bus']
truck_mn['dm_MADT_singleunit'] = truck_mn['ndays_month'] * truck_mn['MADT_singleunit']
truck_mn['dm_MADT_combination'] = truck_mn['ndays_month'] * truck_mn['MADT_combination']
truck_mn['dm_MADT_Truck_Count'] = truck_mn['ndays_month'] * truck_mn['MADT_Truck_Count']

truck_mn.head()

,Station,Year,Month,wjm_naxle2,wjm_naxle3,wjm_naxle4,wjm_naxle5,wjm_naxle6,wjm_naxle7,wjm_TotalCount,wjm_motorcycles,wjm_passenger_cars,wjm_other2axle4tire,wjm_bus,wjm_singleunit,wjm_combination,wjm_Truck_Count,ndays_month,MADT_naxle2,MADT_naxle3,MADT_naxle4,MADT_naxle5,MADT_naxle6,MADT_naxle7,MADT_TotalCount,MADT_motorcycles,MADT_passenger_cars,MADT_other2axle4tire,MADT_bus,MADT_singleunit,MADT_combination,MADT_Truck_Count,dm_MADT_naxle2,dm_MADT_naxle3,dm_MADT_naxle4,dm_MADT_naxle5,dm_MADT_naxle6,dm_MADT_naxle7,dm_MADT_TotalCount,dm_MADT_motorcycles,dm_MADT_passenger_cars,dm_MADT_other2axle4tire,dm_MADT_bus,dm_MADT_singleunit,dm_MADT_combination,dm_MADT_Truck_Count
0,001P34,2017,1,77933.000000,443.000000,219.000000,521.0,3.0,0.0,79119.0,184.000000,65886.0,9583.0,318.0,2423.0,725.000000,3148.000000,31,2513.967742,14.290323,7.064516,16.806452,0.096774,0.0,2552.225806,5.935484,2125.354839,309.129032,10.258065,78.161290,23.387097,101.548387,77933.000000,443.000000,219.000000,521.0,3.0,0.0,79119.0,184.000000,65886.0,9583.0,318.0,2423.0,725.000000,3148.000000
1,001P34,2017,2,78357.000000,384.000000,283.000000,599.0,6.0,0.0,79629.0,119.000000,64668.0,11182.0,310.0,2489.0,861.000000,3350.000000,28,2798.464286,13.714286,10.107143,21.392857,0.214286,0.0,2843.892857,4.250000,2309.571429,399.357143,11.071429,88.892857,30.750000,119.642857,78357.000000,384.000000,283.000000,599.0,6.0,0.0,79629.0,119.000000,64668.0,11182.0,310.0,2489.0,861.000000,3350.000000
2,001P34,2017,3,87814.666667,454.666667,267.666667,683.0,1.0,0.0,89221.0,113.333333,71921.0,12972.0,393.0,2901.0,920.666667,3821.666667,31,2832.731183,14.666667,8.634409,22.032258,0.032258,0.0,2878.096774,3.655914,2320.032258,418.451613,12.677419,93.580645,29.698925,123.279570,87814.666667,454.666667,267.666667,683.0,1.0,0.0,89221.0,113.333333,71921.0,12972.0,393.0,2901.0,920.666667,3821.666667
3,001P34,2017,4,86985.000000,549.000000,256.000000,601.0,2.0,0.0,88393.0,170.000000,68095.0,15735.0,356.0,3213.0,824.000000,4037.000000,30,2899.500000,18.300000,8.533333,20.033333,0.066667,0.0,2946.433333,5.666667,2269.833333,524.500000,11.866667,107.100000,27.466667,134.566667,86985.000000,549.000000,256.000000,601.0,2.0,0.0,88393.0,170.000000,68095.0,15735.0,356.0,3213.0,824.000000,4037.000000
4,001P34,2017,5,89642.000000,625.000000,299.000000,631.0,3.0,0.0,91200.0,170.000000,69399.0,16774.0,381.0,3601.0,875.000000,4476.000000,31,2891.677419,20.161290,9.645161,20.354839,0.096774,0.0,2941.935484,5.483871,2238.677419,541.096774,12.290323,116.161290,28.225806,144.387097,89642.000000,625.000000,299.000000,631.0,3.0,0.0,91200.0,170.000000,69399.0,16774.0,381.0,3601.0,875.000000,4476.000000


In [27]:
truck_yr = truck_mn.groupby(['Station','Year'], as_index=False).agg({
    'dm_MADT_naxle2':'sum', 'dm_MADT_naxle3':'sum', 'dm_MADT_naxle4':'sum',
    'dm_MADT_naxle5':'sum', 'dm_MADT_naxle6':'sum', 'dm_MADT_naxle7':'sum',
    
    'dm_MADT_TotalCount':'sum', 'dm_MADT_motorcycles':'sum', 'dm_MADT_passenger_cars':'sum',
    'dm_MADT_other2axle4tire':'sum', 'dm_MADT_bus':'sum', 'dm_MADT_singleunit':'sum',
    'dm_MADT_combination':'sum', 'dm_MADT_Truck_Count':'sum', 'ndays_month':'sum'
})

truck_yr.columns = ['Station', 'Year', 'sum_dm_MADT_naxle2', 'sum_dm_MADT_naxle3', 'sum_dm_MADT_naxle4', 
                    'sum_dm_MADT_naxle5', 'sum_dm_MADT_naxle6', 'sum_dm_MADT_naxle7', 'sum_dm_MADT_TotalCount', 
                    'sum_dm_MADT_motorcycles', 'sum_dm_MADT_passenger_cars','sum_dm_MADT_other2axle4tire', 
                    'sum_dm_MADT_bus', 'sum_dm_MADT_singleunit', 'sum_dm_MADT_combination', 
                    'sum_dm_MADT_Truck_Count', 'ndays_year_with_data']

truck_yr.head()

,Station,Year,sum_dm_MADT_naxle2,sum_dm_MADT_naxle3,sum_dm_MADT_naxle4,sum_dm_MADT_naxle5,sum_dm_MADT_naxle6,sum_dm_MADT_naxle7,sum_dm_MADT_TotalCount,sum_dm_MADT_motorcycles,sum_dm_MADT_passenger_cars,sum_dm_MADT_other2axle4tire,sum_dm_MADT_bus,sum_dm_MADT_singleunit,sum_dm_MADT_combination,sum_dm_MADT_Truck_Count,ndays_year_with_data
0,001P34,2017,7.741198e+05,4045.333333,2751.416667,5439.083333,28.666667,0.000000,7.863842e+05,2064.000000,6.176978e+05,127998.000000,3219.333333,27599.750000,7805.416667,35405.166667,273
1,005P71,2017,2.201658e+06,11222.750000,29730.166667,180553.166667,2547.500000,174.583333,2.425887e+06,6717.333333,1.710999e+06,398805.083333,15960.416667,85304.666667,208100.000000,293404.666667,212
2,005P71,2018,3.709325e+06,21844.583333,83352.833333,386412.833333,5697.083333,487.833333,4.207120e+06,21054.750000,2.650408e+06,836373.750000,43516.916667,186891.333333,468875.666667,655767.000000,365
3,005P71,2019,3.802372e+06,21706.000000,90757.833333,389595.000000,5221.166667,373.666667,4.310026e+06,10150.916667,2.648580e+06,921709.916667,47661.083333,202632.250000,479290.833333,681923.083333,365
4,005P71,2020,3.294238e+06,23799.916667,88860.916667,389678.916667,5306.166667,284.750000,3.802169e+06,8713.416667,2.215546e+06,854572.750000,42231.916667,202721.250000,478383.250000,681104.500000,366


In [28]:
truck_yr['naxle2_AADT'] = truck_yr['sum_dm_MADT_naxle2']/truck_yr['ndays_year_with_data']  
truck_yr['naxle3_AADT'] = truck_yr['sum_dm_MADT_naxle3']/truck_yr['ndays_year_with_data']
truck_yr['naxle4_AADT'] = truck_yr['sum_dm_MADT_naxle4']/truck_yr['ndays_year_with_data']
truck_yr['naxle5_AADT'] = truck_yr['sum_dm_MADT_naxle5']/truck_yr['ndays_year_with_data']
truck_yr['naxle6_AADT'] = truck_yr['sum_dm_MADT_naxle6']/truck_yr['ndays_year_with_data']
truck_yr['naxle7_AADT'] = truck_yr['sum_dm_MADT_naxle7']/truck_yr['ndays_year_with_data']

truck_yr['TotalCount_AADT'] = truck_yr['sum_dm_MADT_TotalCount']/truck_yr['ndays_year_with_data']
truck_yr['motorcycles_AADT'] = truck_yr['sum_dm_MADT_motorcycles']/truck_yr['ndays_year_with_data']
truck_yr['passenger_cars_AADT'] = truck_yr['sum_dm_MADT_passenger_cars']/truck_yr['ndays_year_with_data']
truck_yr['other2axle4tire_AADT'] = truck_yr['sum_dm_MADT_other2axle4tire']/truck_yr['ndays_year_with_data']
truck_yr['bus_AADT'] = truck_yr['sum_dm_MADT_bus']/truck_yr['ndays_year_with_data']
truck_yr['singleunit_AADT'] = truck_yr['sum_dm_MADT_singleunit']/truck_yr['ndays_year_with_data']
truck_yr['combination_AADT'] = truck_yr['sum_dm_MADT_combination']/truck_yr['ndays_year_with_data']
truck_yr['Truck_Count_AADT'] = truck_yr['sum_dm_MADT_Truck_Count']/truck_yr['ndays_year_with_data']

In [29]:
truck_yr = truck_yr[['Station', 'Year','TotalCount_AADT','naxle2_AADT', 'naxle3_AADT', 'naxle4_AADT', 
                     'naxle5_AADT','naxle6_AADT', 'naxle7_AADT','motorcycles_AADT','passenger_cars_AADT', 
                     'other2axle4tire_AADT', 'bus_AADT','singleunit_AADT', 'combination_AADT', 'Truck_Count_AADT']]

truck_yr.head()

,Station,Year,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT
0,001P34,2017,2880.528388,2835.603480,14.818071,10.078449,19.923382,0.105006,0.000000,7.560440,2262.629121,468.857143,11.792430,101.097985,28.591270,129.689255
1,005P71,2017,11442.861242,10385.181211,52.937500,140.236635,851.665881,12.016509,0.823506,31.685535,8070.750393,1881.156053,75.284984,402.380503,981.603774,1383.984277
2,005P71,2018,11526.355936,10162.533562,59.848174,228.363927,1058.665297,15.608447,1.336530,57.684247,7261.390411,2291.434932,119.224429,512.031050,1284.590868,1796.621918
3,005P71,2019,11808.289041,10417.457078,59.468493,248.651598,1067.383562,14.304566,1.023744,27.810731,7256.384932,2525.232648,130.578311,555.156849,1313.125571,1868.282420
4,005P71,2020,10388.439663,9000.650956,65.027095,242.789390,1064.696494,14.497723,0.778005,23.807149,6053.405282,2334.898224,115.387750,553.883197,1307.058060,1860.941257


In [30]:
# (totalvehcount * 2 axles)/ total axles where total axles = (veh class * axles)
truck_yr['total_axles'] = ((truck_yr['naxle2_AADT']*2)+(truck_yr['naxle3_AADT']*3)+(truck_yr['naxle4_AADT']*4)+
                           (truck_yr['naxle5_AADT']*5)+(truck_yr['naxle6_AADT']*6)+(truck_yr['naxle7_AADT']*7))

truck_yr['axle_factor'] = (truck_yr['TotalCount_AADT'] * 2)/(truck_yr['total_axles'])
truck_yr.head()

,Station,Year,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor
0,001P34,2017,2880.528388,2835.603480,14.818071,10.078449,19.923382,0.105006,0.000000,7.560440,2262.629121,468.857143,11.792430,101.097985,28.591270,129.689255,5856.221917,0.983750
1,005P71,2017,11442.861242,10385.181211,52.937500,140.236635,851.665881,12.016509,0.823506,31.685535,8070.750393,1881.156053,75.284984,402.380503,981.603774,1383.984277,25826.314465,0.886140
2,005P71,2018,11526.355936,10162.533562,59.848174,228.363927,1058.665297,15.608447,1.336530,57.684247,7261.390411,2291.434932,119.224429,512.031050,1284.590868,1796.621918,26814.400228,0.859714
3,005P71,2019,11808.289041,10417.457078,59.468493,248.651598,1067.383562,14.304566,1.023744,27.810731,7256.384932,2525.232648,130.578311,555.156849,1313.125571,1868.282420,27437.837443,0.860730
4,005P71,2020,10388.439663,9000.650956,65.027095,242.789390,1064.696494,14.497723,0.778005,23.807149,6053.405282,2334.898224,115.387750,553.883197,1307.058060,1860.941257,24583.455601,0.845157


In [31]:
truck_yr = pd.merge(stations, truck_yr, on=['Station','Year'], how='right')
truck_yr.head(3)

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor
0,001P34,Adair,KY-0080,20.05,7,5,2017.0,67.671233,2880.528388,2835.603480,14.818071,10.078449,19.923382,0.105006,0.000000,7.560440,2262.629121,468.857143,11.792430,101.097985,28.591270,129.689255,5856.221917,0.983750
1,005P71,Barren,LN-9008,10.10,12,2,2017.0,53.150685,11442.861242,10385.181211,52.937500,140.236635,851.665881,12.016509,0.823506,31.685535,8070.750393,1881.156053,75.284984,402.380503,981.603774,1383.984277,25826.314465,0.886140
2,005P71,Barren,LN-9008,10.10,12,2,2018.0,93.698630,11526.355936,10162.533562,59.848174,228.363927,1058.665297,15.608447,1.336530,57.684247,7261.390411,2291.434932,119.224429,512.031050,1284.590868,1796.621918,26814.400228,0.859714


### Truck Peak Hour Volume

In [32]:
truck_agg_hourly = pd.merge(truck_agg_hr, wjm, on=['Year','Month','dow'], how='left')
truck_agg_hourly.head()

,Station,Year,Month,dow,Hour,TotalCount,naxle2,naxle3,naxle4,naxle5,naxle6,naxle7,motorcycles,passenger_cars,other2axle4tire,bus,singleunit,combination,Truck_Count,isweekend,ndayswithdata,typicalhr_naxle2,typicalhr_naxle3,typicalhr_naxle4,typicalhr_naxle5,typicalhr_naxle6,typicalhr_naxle7,typicalhr_totalcount,typicalhr_motorcycles,typicalhr_passenger_cars,typicalhr_other2axle4tire,typicalhr_bus,typicalhr_singleunit,typicalhr_combination,typicalhr_Truck_Count,ndays_month,nwkdays,nwkends,W_jm
0,001P34,2017,1,0,0,41,41,0,0,0,0,0,0,36,4,0,1,0,1,0,5,8.2,0.0,0.0,0.0,0.0,0.0,8.2,0.0,7.2,0.8,0.0,0.2,0.0,0.2,31,22,9,5
1,001P34,2017,1,0,1,26,26,0,0,0,0,0,0,25,0,0,1,0,1,0,5,5.2,0.0,0.0,0.0,0.0,0.0,5.2,0.0,5.0,0.0,0.0,0.2,0.0,0.2,31,22,9,5
2,001P34,2017,1,0,2,40,39,0,0,1,0,0,0,36,1,0,2,1,3,0,5,7.8,0.0,0.0,0.2,0.0,0.0,8.0,0.0,7.2,0.2,0.0,0.4,0.2,0.6,31,22,9,5
3,001P34,2017,1,0,3,106,106,0,0,0,0,0,1,91,13,0,1,0,1,0,5,21.2,0.0,0.0,0.0,0.0,0.0,21.2,0.2,18.2,2.6,0.0,0.2,0.0,0.2,31,22,9,5
4,001P34,2017,1,0,4,128,126,0,1,1,0,0,0,106,19,0,1,2,3,0,5,25.2,0.0,0.2,0.2,0.0,0.0,25.6,0.0,21.2,3.8,0.0,0.2,0.4,0.6,31,22,9,5


In [33]:
truck_agg_hourly['wjm_TotalCount'] = truck_agg_hourly['W_jm'] * truck_agg_hourly['typicalhr_totalcount']
truck_agg_hourly['wjm_Truck_Count'] = truck_agg_hourly['W_jm'] * truck_agg_hourly['typicalhr_Truck_Count']
truck_agg_hourly.head()

,Station,Year,Month,dow,Hour,TotalCount,naxle2,naxle3,naxle4,naxle5,naxle6,naxle7,motorcycles,passenger_cars,other2axle4tire,bus,singleunit,combination,Truck_Count,isweekend,ndayswithdata,typicalhr_naxle2,typicalhr_naxle3,typicalhr_naxle4,typicalhr_naxle5,typicalhr_naxle6,typicalhr_naxle7,typicalhr_totalcount,typicalhr_motorcycles,typicalhr_passenger_cars,typicalhr_other2axle4tire,typicalhr_bus,typicalhr_singleunit,typicalhr_combination,typicalhr_Truck_Count,ndays_month,nwkdays,nwkends,W_jm,wjm_TotalCount,wjm_Truck_Count
0,001P34,2017,1,0,0,41,41,0,0,0,0,0,0,36,4,0,1,0,1,0,5,8.2,0.0,0.0,0.0,0.0,0.0,8.2,0.0,7.2,0.8,0.0,0.2,0.0,0.2,31,22,9,5,41.0,1.0
1,001P34,2017,1,0,1,26,26,0,0,0,0,0,0,25,0,0,1,0,1,0,5,5.2,0.0,0.0,0.0,0.0,0.0,5.2,0.0,5.0,0.0,0.0,0.2,0.0,0.2,31,22,9,5,26.0,1.0
2,001P34,2017,1,0,2,40,39,0,0,1,0,0,0,36,1,0,2,1,3,0,5,7.8,0.0,0.0,0.2,0.0,0.0,8.0,0.0,7.2,0.2,0.0,0.4,0.2,0.6,31,22,9,5,40.0,3.0
3,001P34,2017,1,0,3,106,106,0,0,0,0,0,1,91,13,0,1,0,1,0,5,21.2,0.0,0.0,0.0,0.0,0.0,21.2,0.2,18.2,2.6,0.0,0.2,0.0,0.2,31,22,9,5,106.0,1.0
4,001P34,2017,1,0,4,128,126,0,1,1,0,0,0,106,19,0,1,2,3,0,5,25.2,0.0,0.2,0.2,0.0,0.0,25.6,0.0,21.2,3.8,0.0,0.2,0.4,0.6,31,22,9,5,128.0,3.0


In [34]:
#sum across dow. i.e. weighted for number of each dow in each month
truck_madt_hr = truck_agg_hourly.groupby(['Station','Year', 'Month', 'Hour'], as_index=False).agg({
                                    'ndays_month':unique_value, 'wjm_TotalCount':'sum', 'wjm_Truck_Count':'sum',   
})

truck_madt_hr['MADThr_allveh'] = truck_madt_hr['wjm_TotalCount']/truck_madt_hr['ndays_month']
truck_madt_hr['MADThr_Truck'] = truck_madt_hr['wjm_Truck_Count']/truck_madt_hr['ndays_month']

truck_madt_hr['dm_MADThr_allveh'] = truck_madt_hr['MADThr_allveh'] * truck_madt_hr['ndays_month'] 
truck_madt_hr['dm_MADThr_Truck'] = truck_madt_hr['MADThr_Truck'] * truck_madt_hr['ndays_month'] 
truck_madt_hr.head()

,Station,Year,Month,Hour,ndays_month,wjm_TotalCount,wjm_Truck_Count,MADThr_allveh,MADThr_Truck,dm_MADThr_allveh,dm_MADThr_Truck
0,001P34,2017,1,0,31,408.0,13.0,13.161290,0.419355,408.0,13.0
1,001P34,2017,1,1,31,290.0,7.0,9.354839,0.225806,290.0,7.0
2,001P34,2017,1,2,31,321.0,9.0,10.354839,0.290323,321.0,9.0
3,001P34,2017,1,3,31,574.0,11.0,18.516129,0.354839,574.0,11.0
4,001P34,2017,1,4,31,853.0,21.0,27.516129,0.677419,853.0,21.0


In [35]:
truck_aadt_hr = truck_madt_hr.groupby(['Station','Year', 'Hour'], as_index=False).agg({
    'dm_MADThr_allveh':'sum', 'dm_MADThr_Truck':'sum', 'ndays_month':'sum'
})

truck_aadt_hr['hourlyAADT_allveh'] = truck_aadt_hr['dm_MADThr_allveh']/truck_aadt_hr['ndays_month']
truck_aadt_hr['hourlyAADT_truck'] = truck_aadt_hr['dm_MADThr_Truck']/truck_aadt_hr['ndays_month']
truck_aadt_hr['ind'] = truck_aadt_hr.index
truck_aadt_hr.head()

,Station,Year,Hour,dm_MADThr_allveh,dm_MADThr_Truck,ndays_month,hourlyAADT_allveh,hourlyAADT_truck,ind
0,001P34,2017,0,3506.750000,100.666667,273,12.845238,0.368742,0
1,001P34,2017,1,2549.833333,79.166667,273,9.340049,0.289988,1
2,001P34,2017,2,2968.166667,132.000000,273,10.872405,0.483516,2
3,001P34,2017,3,5009.250000,194.750000,273,18.348901,0.713370,3
4,001P34,2017,4,8645.416667,400.750000,273,31.668193,1.467949,4


In [36]:
#highest hourly AADT volume and its index
pk_hvol = lambda x: x.nlargest(1).iloc[0]
pk_hvol_index = lambda x: x.nlargest(1).index[0]  

In [37]:
## for each station-year get hour of day with highest all vehicle volume (highest AADThr)


peakhr = truck_aadt_hr.groupby(['Station', 'Year'], as_index=False).agg({'hourlyAADT_allveh':[pk_hvol, pk_hvol_index]})
peakhr.columns = ['Station', 'Year', 'Peakhr_AADT', 'Peakhr_AADT_index']
peakhr.head()

,Station,Year,Peakhr_AADT,Peakhr_AADT_index
0,001P34,2017,256.185287,15
1,005P71,2017,873.935142,40
2,005P71,2018,905.142237,64
3,005P71,2019,945.090411,88
4,005P71,2020,815.637067,112


In [38]:
## then get the truck volume for that hour
truck_peakhr = pd.merge(peakhr, truck_aadt_hr[['ind','Station','Year','Hour','hourlyAADT_allveh','hourlyAADT_truck']],
                        left_on=['Station', 'Year','Peakhr_AADT_index'],
                        right_on = ['Station', 'Year','ind'])

truck_peakhr.columns = ['Station','Year','Peakhr_AADT','Peakhr_AADT_index','ind','Hour','hourlyAADT_allveh', 'Peakhr_truck']

truck_peakhr.head()

,Station,Year,Peakhr_AADT,Peakhr_AADT_index,ind,Hour,hourlyAADT_allveh,Peakhr_truck
0,001P34,2017,256.185287,15,15,15,256.185287,8.985043
1,005P71,2017,873.935142,40,40,16,873.935142,73.317610
2,005P71,2018,905.142237,64,64,16,905.142237,102.726712
3,005P71,2019,945.090411,88,88,16,945.090411,103.518493
4,005P71,2020,815.637067,112,112,16,815.637067,101.962204


In [39]:
#check
len(truck_peakhr[(truck_peakhr.Peakhr_AADT != truck_peakhr.hourlyAADT_allveh)])

0

In [40]:
truck_peak_stats = truck_peakhr[['Station', 'Year','Peakhr_AADT', 'Peakhr_truck', ]]
truck_peak_stats.columns = ['Station', 'Year','Peakhr_AADT', 'Peakhr_Truck_Count',]
truck_peak_stats.head()

,Station,Year,Peakhr_AADT,Peakhr_Truck_Count
0,001P34,2017,256.185287,8.985043
1,005P71,2017,873.935142,73.317610
2,005P71,2018,905.142237,102.726712
3,005P71,2019,945.090411,103.518493
4,005P71,2020,815.637067,101.962204


In [41]:
truck_stats = pd.merge(truck_yr, truck_peak_stats, on=['Station','Year'], how='left')

#Dr.Chen: “peak hour truck %” is defined as truck volume during the peak (total volume) hour of the day divided by that peak hour (total) volume.  

truck_stats['Peakhour_truck_percentage'] = truck_stats['Peakhr_Truck_Count']*100/truck_stats['Peakhr_AADT']
truck_stats.head()

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_AADT,Peakhr_Truck_Count,Peakhour_truck_percentage
0,001P34,Adair,KY-0080,20.05,7,5,2017.0,67.671233,2880.528388,2835.603480,14.818071,10.078449,19.923382,0.105006,0.000000,7.560440,2262.629121,468.857143,11.792430,101.097985,28.591270,129.689255,5856.221917,0.983750,256.185287,8.985043,3.507244
1,005P71,Barren,LN-9008,10.10,12,2,2017.0,53.150685,11442.861242,10385.181211,52.937500,140.236635,851.665881,12.016509,0.823506,31.685535,8070.750393,1881.156053,75.284984,402.380503,981.603774,1383.984277,25826.314465,0.886140,873.935142,73.317610,8.389365
2,005P71,Barren,LN-9008,10.10,12,2,2018.0,93.698630,11526.355936,10162.533562,59.848174,228.363927,1058.665297,15.608447,1.336530,57.684247,7261.390411,2291.434932,119.224429,512.031050,1284.590868,1796.621918,26814.400228,0.859714,905.142237,102.726712,11.349234
3,005P71,Barren,LN-9008,10.10,12,2,2019.0,97.534247,11808.289041,10417.457078,59.468493,248.651598,1067.383562,14.304566,1.023744,27.810731,7256.384932,2525.232648,130.578311,555.156849,1313.125571,1868.282420,27437.837443,0.860730,945.090411,103.518493,10.953290
4,005P71,Barren,LN-9008,10.10,12,2,2020.0,98.087432,10388.439663,9000.650956,65.027095,242.789390,1064.696494,14.497723,0.778005,23.807149,6053.405282,2334.898224,115.387750,553.883197,1307.058060,1860.941257,24583.455601,0.845157,815.637067,101.962204,12.500928


In [42]:
truck_stats['motorcycle_percentage'] = truck_stats['motorcycles_AADT']*100/truck_stats['TotalCount_AADT']
truck_stats['passenger_cars_percentage'] = truck_stats['passenger_cars_AADT']*100/truck_stats['TotalCount_AADT']
truck_stats['other2axle4tire_percentage'] = truck_stats['other2axle4tire_AADT']*100/truck_stats['TotalCount_AADT']
truck_stats['bus_percentage'] = truck_stats['bus_AADT']*100/truck_stats['TotalCount_AADT']
truck_stats['singleunit_percentage'] = truck_stats['singleunit_AADT']*100/truck_stats['TotalCount_AADT']
truck_stats['combination_percentage'] = truck_stats['combination_AADT']*100/truck_stats['TotalCount_AADT']
truck_stats['truck_percentage'] = truck_stats['Truck_Count_AADT']*100/truck_stats['TotalCount_AADT']

In [43]:
truck_stats.head()

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_AADT,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,001P34,Adair,KY-0080,20.05,7,5,2017.0,67.671233,2880.528388,2835.603480,14.818071,10.078449,19.923382,0.105006,0.000000,7.560440,2262.629121,468.857143,11.792430,101.097985,28.591270,129.689255,5856.221917,0.983750,256.185287,8.985043,3.507244,0.262467,78.549100,16.276776,0.409384,3.509703,0.992570,4.502273
1,005P71,Barren,LN-9008,10.10,12,2,2017.0,53.150685,11442.861242,10385.181211,52.937500,140.236635,851.665881,12.016509,0.823506,31.685535,8070.750393,1881.156053,75.284984,402.380503,981.603774,1383.984277,25826.314465,0.886140,873.935142,73.317610,8.389365,0.276902,70.530877,16.439560,0.657921,3.516433,8.578307,12.094740
2,005P71,Barren,LN-9008,10.10,12,2,2018.0,93.698630,11526.355936,10162.533562,59.848174,228.363927,1058.665297,15.608447,1.336530,57.684247,7261.390411,2291.434932,119.224429,512.031050,1284.590868,1796.621918,26814.400228,0.859714,905.142237,102.726712,11.349234,0.500455,62.998145,19.879960,1.034364,4.442263,11.144813,15.587077
3,005P71,Barren,LN-9008,10.10,12,2,2019.0,97.534247,11808.289041,10417.457078,59.468493,248.651598,1067.383562,14.304566,1.023744,27.810731,7256.384932,2525.232648,130.578311,555.156849,1313.125571,1868.282420,27437.837443,0.860730,945.090411,103.518493,10.953290,0.235519,61.451620,21.385254,1.105819,4.701416,11.120371,15.821788
4,005P71,Barren,LN-9008,10.10,12,2,2020.0,98.087432,10388.439663,9000.650956,65.027095,242.789390,1064.696494,14.497723,0.778005,23.807149,6053.405282,2334.898224,115.387750,553.883197,1307.058060,1860.941257,24583.455601,0.845157,815.637067,101.962204,12.500928,0.229170,58.270592,22.475928,1.110732,5.331727,12.581852,17.913578


### aggregation to FC

In [44]:
truck_stats_filtered = truck_stats.loc[(truck_stats.Year >= 2019) & (truck_stats.year_percent_clean_data>=25)]
truck_stats_filtered.head()

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_AADT,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
3,005P71,Barren,LN-9008,10.10,12,2,2019.0,97.534247,11808.289041,10417.457078,59.468493,248.651598,1067.383562,14.304566,1.023744,27.810731,7256.384932,2525.232648,130.578311,555.156849,1313.125571,1868.282420,27437.837443,0.860730,945.090411,103.518493,10.953290,0.235519,61.451620,21.385254,1.105819,4.701416,11.120371,15.821788
4,005P71,Barren,LN-9008,10.10,12,2,2020.0,98.087432,10388.439663,9000.650956,65.027095,242.789390,1064.696494,14.497723,0.778005,23.807149,6053.405282,2334.898224,115.387750,553.883197,1307.058060,1860.941257,24583.455601,0.845157,815.637067,101.962204,12.500928,0.229170,58.270592,22.475928,1.110732,5.331727,12.581852,17.913578
5,005P71,Barren,LN-9008,10.10,12,2,2021.0,96.438356,11916.879224,10416.363927,73.807306,287.695890,1122.886530,15.035845,1.089726,32.574658,7056.280137,2637.353881,142.013242,649.867123,1398.790183,2048.657306,27917.209132,0.853730,942.758904,112.826027,11.967644,0.273349,59.212483,22.131246,1.191698,5.453333,11.737890,17.191223
8,009P26,Bourbon,US-0068,3.85,14,3,2019.0,94.246575,8294.884703,7960.514612,39.384247,81.350913,208.315982,4.930137,0.388813,23.832648,5422.808219,2032.841096,48.930137,484.055936,282.416667,766.472603,17438.468037,0.951332,743.507306,52.556393,7.068712,0.287317,65.375330,24.507165,0.589883,5.835596,3.404709,9.240304
9,009P26,Bourbon,US-0068,3.85,14,3,2020.0,99.453552,7655.547814,7318.500455,47.252277,87.024362,199.220856,3.229281,0.320583,21.889344,4836.238388,1983.189208,42.963115,499.096995,272.170765,771.267760,16144.579235,0.948374,688.271175,54.290984,7.888022,0.285928,63.172989,25.905255,0.561202,6.519416,3.555210,10.074625


In [45]:
#aggregate truck stats by stations across years 
truck_stations = truck_stats_filtered.groupby(['Station','FC_Old','FC_New'], as_index=False).agg({'TotalCount_AADT':'mean',
                                                'naxle2_AADT':'mean','naxle3_AADT':'mean','naxle4_AADT':'mean',
                                                'naxle5_AADT':'mean','naxle6_AADT':'mean','naxle7_AADT':'mean',   
                                                'motorcycles_AADT':'mean','passenger_cars_AADT':'mean',
                                                'other2axle4tire_AADT':'mean','bus_AADT':'mean',
                                                'singleunit_AADT':'mean','combination_AADT':'mean', 
                                                'Truck_Count_AADT':'mean','total_axles':'mean','axle_factor':'mean',
                                                'Peakhour_truck_percentage':'mean', 'Peakhr_Truck_Count':'mean',
                                                'motorcycle_percentage':'mean','passenger_cars_percentage':'mean',
                                                'other2axle4tire_percentage':'mean','bus_percentage':'mean',
                                                'singleunit_percentage':'mean','combination_percentage':'mean',
                                                'truck_percentage':'mean'})

truck_stations.head()

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,005P71,12,2,11371.202643,9944.823987,66.100965,259.712293,1084.988862,14.612711,0.963825,28.064179,6788.690117,2499.161585,129.326434,586.302390,1339.657938,1925.960328,26646.167392,0.853206,11.807287,106.102242,0.246012,59.644899,21.997476,1.136083,5.162159,11.813371,16.975530
1,009P26,14,3,8055.578571,7718.045662,42.947410,86.952109,203.150864,4.111358,0.371168,23.454058,5142.659067,2048.660786,50.663961,509.670535,280.470164,790.140699,16955.762637,0.950144,7.651447,54.940069,0.291046,63.819660,25.445302,0.627514,6.332872,3.483607,9.816479
2,010I01,14,3,12121.719747,11590.988999,62.011276,94.347360,355.074257,18.658966,0.638889,29.197745,7834.881738,3027.174092,78.798955,697.022002,454.645215,1151.667217,25637.198570,0.945635,6.891987,74.887789,0.240871,64.635067,24.973140,0.650064,5.750191,3.750666,9.500857
3,013I02,8,6,324.801296,320.095858,2.314192,1.428144,0.742384,0.218895,0.001821,3.587208,213.838090,89.695974,1.095083,14.414342,2.170600,16.584942,657.884915,0.987471,4.771079,1.335282,1.104441,65.825689,27.632613,0.338329,4.433940,0.664987,5.098928
4,016P37,7,5,1978.343765,1929.007104,11.409670,11.860948,25.758491,0.289318,0.018235,11.901827,1348.504493,476.644183,10.198236,93.883780,37.211245,131.095025,4070.343012,0.972094,5.446237,8.923377,0.601082,68.147304,24.109469,0.515280,4.746858,1.880007,6.626865


In [46]:
#aggregate stats by functional class across stations for old fc
oldfc_truck = truck_stations.groupby(['FC_Old'], as_index=False).agg({'Station':'nunique','TotalCount_AADT':'mean',
                                                'naxle2_AADT':'mean','naxle3_AADT':'mean','naxle4_AADT':'mean',
                                                'naxle5_AADT':'mean','naxle6_AADT':'mean','naxle7_AADT':'mean',   
                                                'motorcycles_AADT':'mean','passenger_cars_AADT':'mean',
                                                'other2axle4tire_AADT':'mean','bus_AADT':'mean',
                                                'singleunit_AADT':'mean','combination_AADT':'mean', 
                                                'Truck_Count_AADT':'mean','total_axles':'mean','axle_factor':'mean',
                                                'Peakhr_Truck_Count':'mean','Peakhour_truck_percentage':'mean', 
                                                'motorcycle_percentage':'mean','passenger_cars_percentage':'mean',
                                                'other2axle4tire_percentage':'mean','bus_percentage':'mean',
                                                'singleunit_percentage':'mean','combination_percentage':'mean',
                                                'truck_percentage':'mean'})
oldfc_truck.head()

,FC_Old,Station,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,1,4,30493.407951,24562.500687,233.715199,749.154032,4728.238018,204.729412,15.070602,73.653762,18311.645697,4809.607785,305.916990,1323.738817,5668.844900,6992.583717,77797.823881,0.779088,414.431058,18.818665,0.238275,58.643402,16.155394,0.989060,4.557939,19.415931,23.973870
1,2,12,11939.559807,11115.522554,95.095722,148.983883,551.277742,27.320874,1.359032,44.706882,7870.960889,2637.369274,85.473461,585.536932,715.512369,1301.049301,26042.094983,0.911384,82.012043,9.329666,0.435864,64.172394,22.625671,0.772637,5.278189,6.715246,11.993435
2,6,7,3541.089383,3360.415652,27.137029,46.416502,86.245179,15.011357,5.863664,20.656961,2204.036849,910.851826,33.562149,227.965756,144.015843,371.981598,7550.248085,0.941534,24.626940,8.009418,0.553332,62.114879,25.918769,0.915808,6.485875,4.011335,10.497210
3,7,7,2387.374162,2350.240500,9.618094,13.235096,12.936233,1.223740,0.120500,12.576488,1611.636270,608.187882,16.797200,112.180006,25.996318,138.176323,4855.142769,0.979132,9.877023,5.518662,0.676301,64.700275,27.356905,0.684223,5.206527,1.375769,6.582296
4,8,3,577.704834,571.641177,1.965961,2.464052,1.532273,0.098635,0.002736,2.899529,373.775295,161.072955,3.047224,33.042230,3.867600,36.909830,1167.308774,0.986758,3.173054,6.211212,0.668497,65.390550,27.246341,0.494722,5.328579,0.871311,6.199890


In [47]:
oldfc_truck

,FC_Old,Station,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,1,4,30493.407951,24562.500687,233.715199,749.154032,4728.238018,204.729412,15.070602,73.653762,18311.645697,4809.607785,305.916990,1323.738817,5668.844900,6992.583717,77797.823881,0.779088,414.431058,18.818665,0.238275,58.643402,16.155394,0.989060,4.557939,19.415931,23.973870
1,2,12,11939.559807,11115.522554,95.095722,148.983883,551.277742,27.320874,1.359032,44.706882,7870.960889,2637.369274,85.473461,585.536932,715.512369,1301.049301,26042.094983,0.911384,82.012043,9.329666,0.435864,64.172394,22.625671,0.772637,5.278189,6.715246,11.993435
2,6,7,3541.089383,3360.415652,27.137029,46.416502,86.245179,15.011357,5.863664,20.656961,2204.036849,910.851826,33.562149,227.965756,144.015843,371.981598,7550.248085,0.941534,24.626940,8.009418,0.553332,62.114879,25.918769,0.915808,6.485875,4.011335,10.497210
3,7,7,2387.374162,2350.240500,9.618094,13.235096,12.936233,1.223740,0.120500,12.576488,1611.636270,608.187882,16.797200,112.180006,25.996318,138.176323,4855.142769,0.979132,9.877023,5.518662,0.676301,64.700275,27.356905,0.684223,5.206527,1.375769,6.582296
4,8,3,577.704834,571.641177,1.965961,2.464052,1.532273,0.098635,0.002736,2.899529,373.775295,161.072955,3.047224,33.042230,3.867600,36.909830,1167.308774,0.986758,3.173054,6.211212,0.668497,65.390550,27.246341,0.494722,5.328579,0.871311,6.199890
5,11,6,81528.855848,70721.309819,425.038865,1236.596746,8763.484403,359.687365,22.738650,218.227518,54333.681697,12860.073299,729.905823,3046.975794,10339.991717,13386.967511,193798.839973,0.840321,766.278561,13.802859,0.268717,65.176663,16.257802,0.880743,3.822033,13.594042,17.416075
6,12,3,35268.328942,33720.745159,189.975428,379.343071,926.326431,44.682977,7.255876,51.922449,25542.270236,6772.008358,190.563491,1391.156258,1320.408151,2711.564409,74479.310031,0.921692,169.618742,7.520642,0.181714,68.460253,20.420273,0.689580,4.280548,5.967633,10.248180
7,14,4,10686.454293,10360.285078,37.429814,73.515279,190.488545,24.390439,0.345138,27.496665,7333.559045,2482.866746,58.556032,504.252590,279.723215,783.975805,22228.122039,0.957044,52.350040,5.942853,0.248676,67.430485,23.961863,0.583264,4.838561,2.937151,7.775712
8,16,4,11522.288995,11405.896476,35.053139,44.077285,35.820692,1.361036,0.080367,32.585073,8422.139174,2508.369553,83.020347,404.466836,71.708011,476.174847,23281.093755,0.984053,31.494127,3.761219,0.341105,71.274876,22.472692,0.734034,4.384500,0.792794,5.177294
9,17,1,1523.050891,1490.680306,10.206172,4.137285,4.325816,13.677406,0.023905,4.959638,994.709829,425.831462,7.307002,68.247853,21.995106,90.242960,3132.389124,0.972434,5.488720,4.061269,0.325786,65.276476,27.988325,0.477399,4.487901,1.444112,5.932013


In [48]:
#aggregate by functional class across stations for new fc
newfc_truck = truck_stations.groupby(['FC_New'], as_index=False).agg({'Station':'nunique','TotalCount_AADT':'mean',
                                                'naxle2_AADT':'mean','naxle3_AADT':'mean','naxle4_AADT':'mean',
                                                'naxle5_AADT':'mean','naxle6_AADT':'mean','naxle7_AADT':'mean',   
                                                'motorcycles_AADT':'mean','passenger_cars_AADT':'mean',
                                                'other2axle4tire_AADT':'mean','bus_AADT':'mean',
                                                'singleunit_AADT':'mean','combination_AADT':'mean', 
                                                'Truck_Count_AADT':'mean','total_axles':'mean','axle_factor':'mean',
                                                'Peakhr_Truck_Count':'mean','Peakhour_truck_percentage':'mean', 
                                                'motorcycle_percentage':'mean','passenger_cars_percentage':'mean',
                                                'other2axle4tire_percentage':'mean','bus_percentage':'mean',
                                                'singleunit_percentage':'mean','combination_percentage':'mean',
                                                'truck_percentage':'mean'})
newfc_truck.head()

,FC_New,Station,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,1,10,61114.676689,52257.786166,348.509398,1041.619661,7149.385849,297.704184,19.671431,160.398015,39924.867297,9639.887093,560.310290,2357.681004,8471.532990,10829.213994,147398.433536,0.815828,625.539560,15.809181,0.256540,62.563359,16.216839,0.924070,4.116396,15.922797,20.039193
1,2,7,23077.318216,21521.027427,176.563168,300.408097,1028.232521,46.723797,4.363206,70.696612,16019.911733,4477.925593,148.252735,1004.155406,1356.376137,2360.531543,50225.424575,0.888245,147.871366,10.632369,0.521726,64.938782,20.044663,0.796596,4.960980,8.737253,13.698233
2,3,12,10857.024514,10445.204538,52.071002,93.086687,246.553228,19.366216,0.742844,25.613359,7356.099618,2545.870349,66.152250,515.652873,347.636065,863.288938,22773.132169,0.942679,55.608445,6.988562,0.259845,65.883329,24.025307,0.674772,5.067274,4.089474,9.156748
3,4,11,6443.343787,6286.045042,30.015615,45.565878,67.909002,10.047604,3.760647,24.994456,4465.164967,1491.767363,51.546948,292.147967,117.722086,409.870052,13270.555601,0.956995,27.124099,6.464618,0.476159,65.445787,24.665650,0.849709,5.721739,2.840956,8.562695
4,5,8,2279.333753,2242.795475,9.691604,12.097870,11.859931,2.780448,0.108426,11.624381,1534.520465,585.393329,15.610925,106.688486,25.496166,132.184653,4639.798563,0.978295,9.328485,5.336487,0.632486,64.772300,27.435832,0.658370,5.116698,1.384312,6.501011


In [49]:
newfc_truck

,FC_New,Station,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,1,10,61114.676689,52257.786166,348.509398,1041.619661,7149.385849,297.704184,19.671431,160.398015,39924.867297,9639.887093,560.310290,2357.681004,8471.532990,10829.213994,147398.433536,0.815828,625.539560,15.809181,0.256540,62.563359,16.216839,0.924070,4.116396,15.922797,20.039193
1,2,7,23077.318216,21521.027427,176.563168,300.408097,1028.232521,46.723797,4.363206,70.696612,16019.911733,4477.925593,148.252735,1004.155406,1356.376137,2360.531543,50225.424575,0.888245,147.871366,10.632369,0.521726,64.938782,20.044663,0.796596,4.960980,8.737253,13.698233
2,3,12,10857.024514,10445.204538,52.071002,93.086687,246.553228,19.366216,0.742844,25.613359,7356.099618,2545.870349,66.152250,515.652873,347.636065,863.288938,22773.132169,0.942679,55.608445,6.988562,0.259845,65.883329,24.025307,0.674772,5.067274,4.089474,9.156748
3,4,11,6443.343787,6286.045042,30.015615,45.565878,67.909002,10.047604,3.760647,24.994456,4465.164967,1491.767363,51.546948,292.147967,117.722086,409.870052,13270.555601,0.956995,27.124099,6.464618,0.476159,65.445787,24.665650,0.849709,5.721739,2.840956,8.562695
4,5,8,2279.333753,2242.795475,9.691604,12.097870,11.859931,2.780448,0.108426,11.624381,1534.520465,585.393329,15.610925,106.688486,25.496166,132.184653,4639.798563,0.978295,9.328485,5.336487,0.632486,64.772300,27.435832,0.658370,5.116698,1.384312,6.501011
5,6,3,577.704834,571.641177,1.965961,2.464052,1.532273,0.098635,0.002736,2.899529,373.775295,161.072955,3.047224,33.042230,3.867600,36.909830,1167.308774,0.986758,3.173054,6.211212,0.668497,65.390550,27.246341,0.494722,5.328579,0.871311,6.199890


In [50]:
#Specify column names
cols_old = ['FC_Old', 'numberofstations', 'TotalCount_AADT', 'naxle2_AADT', 'naxle3_AADT',
               'naxle4_AADT', 'naxle5_AADT', 'naxle6_AADT', 'naxle7_AADT','motorcycles_AADT', 
               'passenger_cars_AADT', 'other2axle4tire_AADT','bus_AADT', 'singleunit_AADT', 
               'combination_AADT', 'Truck_AADT','total_axles','axle_factor','Peakhr_Truck_Volume',
               'Peakhour_truck_percentage','motorcycle_percentage', 'passenger_cars_percentage',
               'other2axle4tire_percentage', 'bus_percentage', 'singleunit_percentage','combination_percentage',
               'Truck_Percentage'] 

cols_new = ['FC_New', 'numberofstations', 'TotalCount_AADT', 'naxle2_AADT', 'naxle3_AADT',
               'naxle4_AADT', 'naxle5_AADT', 'naxle6_AADT', 'naxle7_AADT','motorcycles_AADT', 
               'passenger_cars_AADT', 'other2axle4tire_AADT','bus_AADT', 'singleunit_AADT', 
               'combination_AADT', 'Truck_AADT','total_axles','axle_factor','Peakhr_Truck_Volume',
               'Peakhour_truck_percentage','motorcycle_percentage', 'passenger_cars_percentage',
               'other2axle4tire_percentage', 'bus_percentage', 'singleunit_percentage','combination_percentage',
               'Truck_Percentage']

#assign specified column names
oldfc_truck.columns, newfc_truck.columns = cols_old, cols_new

In [51]:
display(oldfc_truck.head())
display(newfc_truck.head())

,FC_Old,numberofstations,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_AADT,total_axles,axle_factor,Peakhr_Truck_Volume,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,Truck_Percentage
0,1,4,30493.407951,24562.500687,233.715199,749.154032,4728.238018,204.729412,15.070602,73.653762,18311.645697,4809.607785,305.916990,1323.738817,5668.844900,6992.583717,77797.823881,0.779088,414.431058,18.818665,0.238275,58.643402,16.155394,0.989060,4.557939,19.415931,23.973870
1,2,12,11939.559807,11115.522554,95.095722,148.983883,551.277742,27.320874,1.359032,44.706882,7870.960889,2637.369274,85.473461,585.536932,715.512369,1301.049301,26042.094983,0.911384,82.012043,9.329666,0.435864,64.172394,22.625671,0.772637,5.278189,6.715246,11.993435
2,6,7,3541.089383,3360.415652,27.137029,46.416502,86.245179,15.011357,5.863664,20.656961,2204.036849,910.851826,33.562149,227.965756,144.015843,371.981598,7550.248085,0.941534,24.626940,8.009418,0.553332,62.114879,25.918769,0.915808,6.485875,4.011335,10.497210
3,7,7,2387.374162,2350.240500,9.618094,13.235096,12.936233,1.223740,0.120500,12.576488,1611.636270,608.187882,16.797200,112.180006,25.996318,138.176323,4855.142769,0.979132,9.877023,5.518662,0.676301,64.700275,27.356905,0.684223,5.206527,1.375769,6.582296
4,8,3,577.704834,571.641177,1.965961,2.464052,1.532273,0.098635,0.002736,2.899529,373.775295,161.072955,3.047224,33.042230,3.867600,36.909830,1167.308774,0.986758,3.173054,6.211212,0.668497,65.390550,27.246341,0.494722,5.328579,0.871311,6.199890


,FC_New,numberofstations,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_AADT,total_axles,axle_factor,Peakhr_Truck_Volume,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,Truck_Percentage
0,1,10,61114.676689,52257.786166,348.509398,1041.619661,7149.385849,297.704184,19.671431,160.398015,39924.867297,9639.887093,560.310290,2357.681004,8471.532990,10829.213994,147398.433536,0.815828,625.539560,15.809181,0.256540,62.563359,16.216839,0.924070,4.116396,15.922797,20.039193
1,2,7,23077.318216,21521.027427,176.563168,300.408097,1028.232521,46.723797,4.363206,70.696612,16019.911733,4477.925593,148.252735,1004.155406,1356.376137,2360.531543,50225.424575,0.888245,147.871366,10.632369,0.521726,64.938782,20.044663,0.796596,4.960980,8.737253,13.698233
2,3,12,10857.024514,10445.204538,52.071002,93.086687,246.553228,19.366216,0.742844,25.613359,7356.099618,2545.870349,66.152250,515.652873,347.636065,863.288938,22773.132169,0.942679,55.608445,6.988562,0.259845,65.883329,24.025307,0.674772,5.067274,4.089474,9.156748
3,4,11,6443.343787,6286.045042,30.015615,45.565878,67.909002,10.047604,3.760647,24.994456,4465.164967,1491.767363,51.546948,292.147967,117.722086,409.870052,13270.555601,0.956995,27.124099,6.464618,0.476159,65.445787,24.665650,0.849709,5.721739,2.840956,8.562695
4,5,8,2279.333753,2242.795475,9.691604,12.097870,11.859931,2.780448,0.108426,11.624381,1534.520465,585.393329,15.610925,106.688486,25.496166,132.184653,4639.798563,0.978295,9.328485,5.336487,0.632486,64.772300,27.435832,0.658370,5.116698,1.384312,6.501011


### aggregate by FClass 

### Table 6

In [52]:
## FC_Old
table_6_oldfc = oldfc_truck[['FC_Old','Truck_AADT','Peakhr_Truck_Volume','Truck_Percentage', 'Peakhour_truck_percentage']]
table_6_oldfc.head()

,FC_Old,Truck_AADT,Peakhr_Truck_Volume,Truck_Percentage,Peakhour_truck_percentage
0,1,6992.583717,414.431058,23.973870,18.818665
1,2,1301.049301,82.012043,11.993435,9.329666
2,6,371.981598,24.626940,10.497210,8.009418
3,7,138.176323,9.877023,6.582296,5.518662
4,8,36.909830,3.173054,6.199890,6.211212


In [53]:
## FC_New
table_6_newfc = newfc_truck[['FC_New','Truck_AADT','Peakhr_Truck_Volume','Truck_Percentage', 'Peakhour_truck_percentage']]
table_6_newfc

,FC_New,Truck_AADT,Peakhr_Truck_Volume,Truck_Percentage,Peakhour_truck_percentage
0,1,10829.213994,625.539560,20.039193,15.809181
1,2,2360.531543,147.871366,13.698233,10.632369
2,3,863.288938,55.608445,9.156748,6.988562
3,4,409.870052,27.124099,8.562695,6.464618
4,5,132.184653,9.328485,6.501011,5.336487
5,6,36.909830,3.173054,6.199890,6.211212


In [54]:
#all Functional classes  combined
truck_allfc_1 = truck_stations.agg({'Truck_Count_AADT':'mean', 
                             'Peakhr_Truck_Count':'mean',
                             'truck_percentage':'mean',
                             'Peakhour_truck_percentage':'mean', 
                             })

truck_allfc_1 = pd.DataFrame(truck_allfc_1).transpose()
truck_allfc_1.columns = ['Truck_AADT','Peakhr_Truck_Volume','Truck_Percentage','Peakhour_truck_percentage']
truck_old, truck_new  = truck_allfc_1.copy(), truck_allfc_1.copy()
truck_old.insert(0, 'FC_Old', '1-17')
truck_new.insert(0, 'FC_New', '1-6')

In [55]:
display(truck_old)
display(truck_new)

,FC_Old,Truck_AADT,Peakhr_Truck_Volume,Truck_Percentage,Peakhour_truck_percentage
0,1-17,2761.805986,163.535463,11.195254,8.800343


,FC_New,Truck_AADT,Peakhr_Truck_Volume,Truck_Percentage,Peakhour_truck_percentage
0,1-6,2761.805986,163.535463,11.195254,8.800343


In [56]:
#add all functional classes combined stats to TFR table 6
table_6_oldfc = pd.concat([table_6_oldfc,truck_old], ignore_index=True)
table_6_newfc = pd.concat([table_6_newfc,truck_new], ignore_index=True)
display(table_6_oldfc)
display(table_6_newfc)

,FC_Old,Truck_AADT,Peakhr_Truck_Volume,Truck_Percentage,Peakhour_truck_percentage
0,1,6992.583717,414.431058,23.973870,18.818665
1,2,1301.049301,82.012043,11.993435,9.329666
2,6,371.981598,24.626940,10.497210,8.009418
3,7,138.176323,9.877023,6.582296,5.518662
4,8,36.909830,3.173054,6.199890,6.211212
5,11,13386.967511,766.278561,17.416075,13.802859
6,12,2711.564409,169.618742,10.248180,7.520642
7,14,783.975805,52.350040,7.775712,5.942853
8,16,476.174847,31.494127,5.177294,3.761219
9,17,90.242960,5.488720,5.932013,4.061269


,FC_New,Truck_AADT,Peakhr_Truck_Volume,Truck_Percentage,Peakhour_truck_percentage
0,1,10829.213994,625.539560,20.039193,15.809181
1,2,2360.531543,147.871366,13.698233,10.632369
2,3,863.288938,55.608445,9.156748,6.988562
3,4,409.870052,27.124099,8.562695,6.464618
4,5,132.184653,9.328485,6.501011,5.336487
5,6,36.909830,3.173054,6.199890,6.211212
6,1-6,2761.805986,163.535463,11.195254,8.800343


### Table 9

In [ ]:
oldfc_truck.head()

In [ ]:
table_9_oldfc = oldfc_truck[['FC_Old','motorcycle_percentage','passenger_cars_percentage','other2axle4tire_percentage',
             'bus_percentage','singleunit_percentage','combination_percentage','axle_factor']]
table_9_oldfc.insert(7, 'All_Vehicles', table_9_oldfc.iloc[:,1:-1].sum(axis=1))
table_9_oldfc.insert(8, 'Heavy_Vehicles', table_9_oldfc.iloc[:,4:7].sum(axis=1))
table_9_oldfc

In [ ]:
table_9_newfc = newfc_truck[['FC_New','motorcycle_percentage','passenger_cars_percentage','other2axle4tire_percentage',
             'bus_percentage','singleunit_percentage','combination_percentage','axle_factor']]
table_9_newfc.insert(7, 'All_Vehicles', table_9_newfc.iloc[:,1:-1].sum(axis=1))
table_9_newfc.insert(8, 'Heavy_Vehicles', table_9_newfc.iloc[:,4:7].sum(axis=1))
table_9_newfc

In [ ]:
#all Functional classes  combined
truck_allfc_2 = truck_stations.agg({'motorcycle_percentage':'mean', 
                             'passenger_cars_percentage':'mean',
                             'other2axle4tire_percentage':'mean',
                             'bus_percentage':'mean', 
                             'singleunit_percentage':'mean', 
                             'combination_percentage':'mean', 
                             'axle_factor':'mean',
                             })

truck_allfc_2 = pd.DataFrame(truck_allfc_2).transpose()
truck_allfc_2.insert(6, 'All_Vehicles', truck_allfc_2.iloc[:,0:-1].sum(axis=1))
truck_allfc_2.insert(7, 'Heavy_Vehicles', truck_allfc_2.iloc[:,3:6].sum(axis=1))
truck_allfc_2

In [ ]:
truck_old1, truck_new1  = truck_allfc_2.copy(), truck_allfc_2.copy()
truck_old1.insert(0, 'FC_Old', '1-17')
truck_new1.insert(0, 'FC_New', '1-6')
display(truck_old1)
display(truck_new1)

In [ ]:
table_9_oldfc = pd.concat([table_9_oldfc,truck_old1], ignore_index=True)
table_9_newfc = pd.concat([table_9_newfc,truck_new1], ignore_index=True)
display(table_9_oldfc)
display(table_9_newfc)

## Aggregate Classification data by AADT classes (<600, between 600-1800, >1800)

In [ ]:
def aadt_group(aadt):
    if aadt < 600:
        aadt_class = 'less_than_600'
    elif 600 <= aadt <= 1800:
        aadt_class = 'between_600_and_1800'
    else:
        aadt_class = 'greather_than_1800'
    return aadt_class

In [ ]:
# Eugene: confirm if all vehicle aadt or truck aadt to be used for grouping
#xu: based on AADTT
truck_stations['aadtt_class'] = truck_stations['Truck_Count_AADT'].map(aadt_group)
truck_stations.head()

In [ ]:
#aggregate stats by aadt class across stations 
aadt_class_stats = truck_stations.groupby(['aadtt_class'], as_index=False).agg({'Station':'nunique','TotalCount_AADT':'mean',
                                                'naxle2_AADT':'mean','naxle3_AADT':'mean','naxle4_AADT':'mean',
                                                'naxle5_AADT':'mean','naxle6_AADT':'mean','naxle7_AADT':'mean',   
                                                'motorcycles_AADT':'mean','passenger_cars_AADT':'mean',
                                                'other2axle4tire_AADT':'mean','bus_AADT':'mean',
                                                'singleunit_AADT':'mean','combination_AADT':'mean', 
                                                'Truck_Count_AADT':'mean','total_axles':'mean','axle_factor':'mean',
                                                'Peakhr_Truck_Count':'mean','Peakhour_truck_percentage':'mean', 
                                                'motorcycle_percentage':'mean','passenger_cars_percentage':'mean',
                                                'other2axle4tire_percentage':'mean','bus_percentage':'mean',
                                                'singleunit_percentage':'mean','combination_percentage':'mean',
                                                'truck_percentage':'mean'})

cols_aadt_class = ['AADTT_Class', 'numberofstations', 'TotalCount_AADT', 'naxle2_AADT', 'naxle3_AADT',
               'naxle4_AADT', 'naxle5_AADT', 'naxle6_AADT', 'naxle7_AADT','motorcycles_AADT', 
               'passenger_cars_AADT', 'other2axle4tire_AADT','bus_AADT', 'singleunit_AADT', 
               'combination_AADT', 'Truck_AADT','total_axles','axle_factor','Peakhr_Truck_Volume',
               'Peakhour_truck_percentage','motorcycle_percentage', 'passenger_cars_percentage',
               'other2axle4tire_percentage', 'bus_percentage', 'singleunit_percentage','combination_percentage',
               'Truck_Percentage'] 

aadt_class_stats.columns = cols_aadt_class
aadt_class_stats.head()

### table 6 for aadt classes

In [ ]:
table6_aadtclass = aadt_class_stats[['AADTT_Class','Truck_AADT','Peakhr_Truck_Volume',
                                     'Truck_Percentage', 'Peakhour_truck_percentage']]
table6_aadtclass

### table 9 for aadt classes

In [ ]:
table9_aadtclass = aadt_class_stats[['AADTT_Class','motorcycle_percentage','passenger_cars_percentage',
                                     'other2axle4tire_percentage','bus_percentage','singleunit_percentage',
                                     'combination_percentage','axle_factor']]
table9_aadtclass.insert(7, 'All_Vehicles', table9_aadtclass.iloc[:,1:-1].sum(axis=1))
table9_aadtclass.insert(8, 'Heavy_Vehicles', table9_aadtclass.iloc[:,4:7].sum(axis=1))
table9_aadtclass

## Checks

In [57]:
###checks
truck_stations.head()

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,005P71,12,2,11371.202643,9944.823987,66.100965,259.712293,1084.988862,14.612711,0.963825,28.064179,6788.690117,2499.161585,129.326434,586.302390,1339.657938,1925.960328,26646.167392,0.853206,11.807287,106.102242,0.246012,59.644899,21.997476,1.136083,5.162159,11.813371,16.975530
1,009P26,14,3,8055.578571,7718.045662,42.947410,86.952109,203.150864,4.111358,0.371168,23.454058,5142.659067,2048.660786,50.663961,509.670535,280.470164,790.140699,16955.762637,0.950144,7.651447,54.940069,0.291046,63.819660,25.445302,0.627514,6.332872,3.483607,9.816479
2,010I01,14,3,12121.719747,11590.988999,62.011276,94.347360,355.074257,18.658966,0.638889,29.197745,7834.881738,3027.174092,78.798955,697.022002,454.645215,1151.667217,25637.198570,0.945635,6.891987,74.887789,0.240871,64.635067,24.973140,0.650064,5.750191,3.750666,9.500857
3,013I02,8,6,324.801296,320.095858,2.314192,1.428144,0.742384,0.218895,0.001821,3.587208,213.838090,89.695974,1.095083,14.414342,2.170600,16.584942,657.884915,0.987471,4.771079,1.335282,1.104441,65.825689,27.632613,0.338329,4.433940,0.664987,5.098928
4,016P37,7,5,1978.343765,1929.007104,11.409670,11.860948,25.758491,0.289318,0.018235,11.901827,1348.504493,476.644183,10.198236,93.883780,37.211245,131.095025,4070.343012,0.972094,5.446237,8.923377,0.601082,68.147304,24.109469,0.515280,4.746858,1.880007,6.626865


In [61]:
oldfc_truck[oldfc_truck.FC_Old.isin([1,11])]

,FC_Old,numberofstations,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_AADT,total_axles,axle_factor,Peakhr_Truck_Volume,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,Truck_Percentage
0,1,4,30493.407951,24562.500687,233.715199,749.154032,4728.238018,204.729412,15.070602,73.653762,18311.645697,4809.607785,305.916990,1323.738817,5668.844900,6992.583717,77797.823881,0.779088,414.431058,18.818665,0.238275,58.643402,16.155394,0.989060,4.557939,19.415931,23.973870
5,11,6,81528.855848,70721.309819,425.038865,1236.596746,8763.484403,359.687365,22.738650,218.227518,54333.681697,12860.073299,729.905823,3046.975794,10339.991717,13386.967511,193798.839973,0.840321,766.278561,13.802859,0.268717,65.176663,16.257802,0.880743,3.822033,13.594042,17.416075


In [60]:
truck_stations[truck_stations.FC_Old == 11]

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
5,019P97,11,1,85369.045991,82719.924135,247.103381,398.118318,1847.710692,152.166274,4.023192,152.971305,67406.919025,12402.399764,496.163522,2541.624214,2368.968160,4910.592374,177953.345126,0.959454,4.608046,341.422563,0.179188,78.959438,14.527982,0.581198,2.977220,2.774973,5.752193
12,034I05,11,1,80744.056345,67875.791915,440.075449,1169.856767,10847.587019,398.759144,11.986049,299.504980,50636.169801,12911.895755,958.339448,3558.459506,12379.686855,15938.146361,198465.629554,0.813512,14.691107,931.087217,0.376376,62.722745,15.941445,1.211439,4.399176,15.348819,19.747995
22,047P68,11,1,61342.926329,45465.863612,411.984835,1389.597343,13560.400385,495.144153,19.936002,103.361776,31859.674915,10665.027175,624.932195,2672.490584,15417.439685,18089.930269,168638.489954,0.726923,24.231595,1069.771002,0.168059,51.829529,17.396131,1.016295,4.360754,25.229232,29.589986
26,056P98,11,1,71047.323880,67251.314181,326.068274,572.692526,2658.806644,203.172508,35.269748,128.242526,49603.843463,14525.287710,425.619288,2960.529552,3403.801341,6364.330893,152531.559789,0.931528,6.618531,390.357019,0.180436,69.797998,20.457505,0.598404,4.169929,4.795728,8.965657
28,059P93,11,1,128651.947989,112871.894252,649.543070,2109.063691,12480.372985,503.292367,37.781625,373.831223,90995.921626,16515.131891,1389.138725,4275.602559,15102.321965,19377.924524,301814.762975,0.851370,11.122295,928.074771,0.302790,70.509378,12.872487,1.094698,3.359808,11.860840,15.220647
47,114P67,11,1,62017.834554,48143.070818,475.458181,1780.251832,11186.028694,405.589744,27.435287,251.453297,35499.561355,10140.697497,485.241758,2273.148352,13367.732295,15640.880647,163389.252442,0.759142,21.545578,936.958791,0.405453,57.240891,16.351260,0.782423,3.665314,21.554658,25.219972


In [91]:
def highlight_rows(row):
    value = row.loc['year_percent_clean_data']
    if value < 25.0:
        color = '#FFB3BA' # Red
    else:
        color = '#ffffff'
    return ['background-color: {}'.format(color) for r in row]

In [92]:
check1 = truck_stats[truck_stats.Station.isin(truck_stations.loc[truck_stations.FC_Old == 1,'Station'])]
check1 = check1.loc[check1.Year >= 2019].reset_index(drop=True)
check1.style.apply(highlight_rows, axis=1)

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_AADT,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,022P47,Carter,I -0064,167.050000,1,1,2019.000000,86.301370,15948.771457,13006.422156,126.398703,335.380489,2369.315120,105.061876,6.193114,32.219810,9395.849551,2793.871008,148.686627,822.810629,2755.333832,3578.144461,40253.861028,0.792410,1212.886228,222.570858,18.350514,0.202021,58.912685,17.517782,0.932276,5.159085,17.276151,22.435236
1,022P47,Carter,I -0064,167.050000,1,1,2020.000000,99.453552,13742.071721,10814.907787,117.580601,386.992486,2329.824454,88.392760,4.373634,27.199454,7489.552596,2573.420082,137.661885,752.952186,2761.285519,3514.237705,35740.621585,0.768989,1044.635246,221.653689,21.218285,0.197928,54.500899,18.726580,1.001755,5.479175,20.093663,25.572838
2,022P47,Carter,I -0064,167.050000,1,1,2021.000000,35.068493,13870.852097,11009.737307,106.925497,341.777042,2340.559603,69.009934,2.842715,13.889625,7661.295254,2563.428256,150.366446,756.910596,2724.961921,3481.872517,35844.115894,0.773954,1080.783664,217.489514,20.123316,0.100135,55.233054,18.480683,1.084046,5.456843,19.645238,25.102081
3,034P74,Fayette,I -0064,71.700000,1,1,2019.000000,30.136986,36415.206111,30670.217222,200.169444,648.020556,4699.524444,186.293889,10.980556,104.805556,23493.249444,5478.817778,418.737778,1405.548889,5514.046667,6919.595556,89225.274444,0.816253,3023.352222,391.293333,12.942367,0.287807,64.514943,15.045412,1.149898,3.859786,15.142154,19.001940
4,072P51,Lyon,I -0024,37.230000,1,1,2019.000000,82.739726,30100.240269,21935.558882,359.893214,1189.908932,6283.456337,301.919661,29.503244,90.871008,15822.320359,4773.672156,231.100798,1382.442365,7799.833583,9182.275948,83145.755489,0.724036,2192.309880,570.937625,26.042743,0.301895,52.565429,15.859249,0.767771,4.592795,25.912862,30.505657
5,072P51,Lyon,I -0024,37.230000,1,1,2020.000000,51.366120,25977.765369,17608.317623,450.370560,1204.449112,6291.262978,376.058743,47.306352,89.775273,12358.951161,4030.907787,195.634221,1397.896516,7904.600410,9302.496926,75429.355191,0.688797,1898.861339,595.813183,31.377393,0.345585,47.575113,15.516761,0.753083,5.381127,30.428331,35.809458
6,106P22,Shelby,I -0064,35.900000,1,1,2019.000000,96.986301,46196.605251,39564.409132,187.260274,773.978311,5456.291781,206.855023,7.810731,78.741096,30685.085160,6898.302511,446.791324,1663.698174,6423.986986,8087.685160,111363.776484,0.829652,3689.079452,440.780822,11.948260,0.170448,66.422814,14.932488,0.967152,3.601343,13.905756,17.507098
7,106P22,Shelby,I -0064,35.900000,1,1,2020.000000,96.994536,38179.645719,31571.614526,208.056922,743.548497,5444.354964,206.476321,5.594490,68.774590,23548.946949,6157.030965,410.665984,1617.762978,6376.464253,7994.227231,95241.387978,0.801745,2988.856330,447.782104,14.981721,0.180134,61.679323,16.126475,1.075615,4.237239,16.701214,20.938453
8,106P22,Shelby,I -0064,35.900000,1,1,2021.000000,96.986301,44620.322374,37456.450913,242.456393,872.572831,5837.857991,205.107763,5.876484,77.634475,28207.363242,7085.917352,480.515753,1883.576256,6885.315297,8768.891553,109591.634247,0.814302,3502.309817,498.889498,14.244585,0.173989,63.216404,15.880471,1.076899,4.221342,15.430895,19.652237


In [96]:
check11 = truck_stats[truck_stats.Station.isin(truck_stations.loc[truck_stations.FC_Old == 11,'Station'])]
check11 = check11.loc[check11.Year >= 2019].reset_index(drop=True)
check11.style.apply(highlight_rows, axis=1)

,Station,County,Route,MP,FC_Old,FC_New,Year,year_percent_clean_data,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhr_AADT,Peakhr_Truck_Count,Peakhour_truck_percentage,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,019P97,Campbell,I -0275,76.350000,11,1,2019.000000,56.164384,85369.045991,82719.924135,247.103381,398.118318,1847.710692,152.166274,4.023192,152.971305,67406.919025,12402.399764,496.163522,2541.624214,2368.968160,4910.592374,177953.345126,0.959454,7409.270047,341.422563,4.608046,0.179188,78.959438,14.527982,0.581198,2.977220,2.774973,5.752193
1,034I05,Fayette,I -0075,116.593000,11,1,2019.000000,1.917808,90469.903226,76674.741935,434.548387,802.451613,12141.354839,408.161290,8.645161,327.677419,58895.870968,13714.677419,901.161290,3314.677419,13315.838710,16630.516129,221079.193548,0.818439,7291.838710,960.354839,13.170270,0.362195,65.099960,15.159381,0.996090,3.663845,14.718529,18.382374
2,034I05,Fayette,I -0075,116.593000,11,1,2020.000000,77.049180,73211.951730,61389.068534,408.171903,1069.726548,9954.476093,380.324681,10.183971,318.280965,46005.295537,11281.015255,1079.619991,3158.644126,11369.095856,14527.739982,180407.175319,0.811630,5806.039162,842.164845,14.504980,0.434739,62.838504,15.408707,1.474650,4.314383,15.529016,19.843399
3,034I05,Fayette,I -0075,116.593000,11,1,2021.000000,82.465753,88276.160959,74362.515297,471.978995,1269.986986,11740.697945,417.193607,13.788128,280.728995,55267.044064,14542.776256,837.058904,3958.274886,13390.277854,17348.552740,216524.083790,0.815393,6856.177397,1020.009589,14.877234,0.318012,62.606986,16.474183,0.948228,4.483968,15.168623,19.652591
4,047P68,Hardin,I -0065,92.215500,11,1,2019.000000,86.575342,63019.404691,47546.055888,373.308134,1234.065369,13341.836327,500.182884,23.956088,110.943114,34028.831088,10681.005739,610.876248,2531.309631,15056.438872,17587.748503,171026.269212,0.736956,4561.710080,1004.127495,22.012085,0.176046,53.997386,16.948757,0.969346,4.016715,23.891750,27.908465
5,047P68,Hardin,I -0065,92.215500,11,1,2020.000000,95.628415,55310.048042,39793.577641,394.407332,1305.435792,13310.338115,489.572177,16.716985,87.507969,27304.731785,9785.755009,544.743852,2501.325820,15085.983607,17587.309426,155598.262978,0.710934,4016.374772,1057.788934,26.336908,0.158214,49.366675,17.692545,0.984891,4.522371,27.275304,31.797675
6,047P68,Hardin,I -0065,92.215500,11,1,2021.000000,97.260274,65699.326256,49057.957306,468.239041,1629.290868,14029.026712,495.677397,19.134932,111.634247,34245.461872,11528.320776,719.176484,2984.836301,16109.896575,19094.732877,179290.937671,0.732879,4712.915525,1147.396575,24.345791,0.169917,52.124525,17.547091,1.094648,4.543176,24.520642,29.063818
7,056P98,Jefferson,I -0265,16.100000,11,1,2019.000000,87.123288,73096.969406,69148.350000,313.318037,581.657078,2736.050228,245.926712,71.667352,141.687900,52161.400685,14101.987900,423.084703,2695.974886,3572.833333,6268.808219,157220.765297,0.929864,6019.235388,369.113014,6.132224,0.193836,71.359184,19.292165,0.578799,3.688217,4.887800,8.576017
8,056P98,Jefferson,I -0265,16.100000,11,1,2020.000000,96.721311,67786.100865,63970.971539,328.823770,569.517304,2692.641849,197.853825,26.292577,121.850638,46864.491348,14138.624089,381.795993,2860.640255,3418.698543,6279.338798,146040.863843,0.928317,5737.412568,394.730647,6.879942,0.179758,69.135842,20.857704,0.563236,4.220099,5.043362,9.263461
9,056P98,Jefferson,I -0265,16.100000,11,1,2021.000000,99.452055,72258.901370,68634.621005,336.063014,566.903196,2547.727854,165.736986,7.849315,121.189041,49785.638356,15335.251142,471.977169,3324.973516,3219.872146,6544.845662,154333.050228,0.936402,5950.634703,407.227397,6.843428,0.167715,68.898969,21.222646,0.653175,4.601

In [72]:
#aggregate truck stats by stations across years 
check1s = check1.groupby(['Station','FC_Old','FC_New'], as_index=False).agg({'TotalCount_AADT':'mean',
                                                'naxle2_AADT':'mean','naxle3_AADT':'mean','naxle4_AADT':'mean',
                                                'naxle5_AADT':'mean','naxle6_AADT':'mean','naxle7_AADT':'mean',   
                                                'motorcycles_AADT':'mean','passenger_cars_AADT':'mean',
                                                'other2axle4tire_AADT':'mean','bus_AADT':'mean',
                                                'singleunit_AADT':'mean','combination_AADT':'mean', 
                                                'Truck_Count_AADT':'mean','total_axles':'mean','axle_factor':'mean',
                                                'Peakhour_truck_percentage':'mean', 'Peakhr_Truck_Count':'mean',
                                                'motorcycle_percentage':'mean','passenger_cars_percentage':'mean',
                                                'other2axle4tire_percentage':'mean','bus_percentage':'mean',
                                                'singleunit_percentage':'mean','combination_percentage':'mean',
                                                'truck_percentage':'mean'})
check1s

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,022P47,1,1,14520.565092,11610.355750,116.968267,354.716672,2346.566392,87.488190,4.469821,24.436296,8182.232467,2643.573115,145.571653,777.557804,2747.193757,3524.751561,37279.532836,0.778451,19.897372,220.571354,0.166695,56.215546,18.241682,1.006026,5.365034,19.005017,24.370051
1,034P74,1,1,36415.206111,30670.217222,200.169444,648.020556,4699.524444,186.293889,10.980556,104.805556,23493.249444,5478.817778,418.737778,1405.548889,5514.046667,6919.595556,89225.274444,0.816253,12.942367,391.293333,0.287807,64.514943,15.045412,1.149898,3.859786,15.142154,19.001940
2,072P51,1,1,28039.002819,19771.938253,405.131887,1197.179022,6287.359658,338.989202,38.404798,90.323141,14090.635760,4402.289971,213.367510,1390.169441,7852.216996,9242.386437,79287.555340,0.706416,28.710068,583.375404,0.323740,50.070271,15.688005,0.760427,4.986961,28.170596,33.157557
3,106P22,1,1,42998.857782,36197.491524,212.591196,796.699880,5579.501578,206.146369,6.427235,75.050054,27480.465117,6713.750276,445.991020,1721.679136,6561.922179,8283.601315,105398.932903,0.815233,13.724855,462.484141,0.174857,63.772847,15.646478,1.039889,4.019975,15.345955,19.365930


In [73]:
truck_stations[truck_stations.FC_Old == 1]

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
7,022P47,1,1,14520.565092,11610.355750,116.968267,354.716672,2346.566392,87.488190,4.469821,24.436296,8182.232467,2643.573115,145.571653,777.557804,2747.193757,3524.751561,37279.532836,0.778451,19.897372,220.571354,0.166695,56.215546,18.241682,1.006026,5.365034,19.005017,24.370051
13,034P74,1,1,36415.206111,30670.217222,200.169444,648.020556,4699.524444,186.293889,10.980556,104.805556,23493.249444,5478.817778,418.737778,1405.548889,5514.046667,6919.595556,89225.274444,0.816253,12.942367,391.293333,0.287807,64.514943,15.045412,1.149898,3.859786,15.142154,19.001940
31,072P51,1,1,28039.002819,19771.938253,405.131887,1197.179022,6287.359658,338.989202,38.404798,90.323141,14090.635760,4402.289971,213.367510,1390.169441,7852.216996,9242.386437,79287.555340,0.706416,28.710068,583.375404,0.323740,50.070271,15.688005,0.760427,4.986961,28.170596,33.157557
44,106P22,1,1,42998.857782,36197.491524,212.591196,796.699880,5579.501578,206.146369,6.427235,75.050054,27480.465117,6713.750276,445.991020,1721.679136,6561.922179,8283.601315,105398.932903,0.815233,13.724855,462.484141,0.174857,63.772847,15.646478,1.039889,4.019975,15.345955,19.365930


In [74]:
#aggregate truck stats by stations across years 
check11s = check11.groupby(['Station','FC_Old','FC_New'], as_index=False).agg({'TotalCount_AADT':'mean',
                                                'naxle2_AADT':'mean','naxle3_AADT':'mean','naxle4_AADT':'mean',
                                                'naxle5_AADT':'mean','naxle6_AADT':'mean','naxle7_AADT':'mean',   
                                                'motorcycles_AADT':'mean','passenger_cars_AADT':'mean',
                                                'other2axle4tire_AADT':'mean','bus_AADT':'mean',
                                                'singleunit_AADT':'mean','combination_AADT':'mean', 
                                                'Truck_Count_AADT':'mean','total_axles':'mean','axle_factor':'mean',
                                                'Peakhour_truck_percentage':'mean', 'Peakhr_Truck_Count':'mean',
                                                'motorcycle_percentage':'mean','passenger_cars_percentage':'mean',
                                                'other2axle4tire_percentage':'mean','bus_percentage':'mean',
                                                'singleunit_percentage':'mean','combination_percentage':'mean',
                                                'truck_percentage':'mean'})
check11s

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,019P97,11,1,85369.045991,82719.924135,247.103381,398.118318,1847.710692,152.166274,4.023192,152.971305,67406.919025,12402.399764,496.163522,2541.624214,2368.968160,4910.592374,177953.345126,0.959454,4.608046,341.422563,0.179188,78.959438,14.527982,0.581198,2.977220,2.774973,5.752193
1,034I05,11,1,83986.005305,70808.775255,438.233095,1047.388382,11278.842959,401.893193,10.872420,308.895793,53389.403523,13179.489643,939.280062,3477.198810,12691.737473,16168.936284,206003.484219,0.815154,14.184162,940.843091,0.371649,63.515150,15.680757,1.139656,4.154066,15.138723,19.292788
2,047P68,11,1,61342.926329,45465.863612,411.984835,1389.597343,13560.400385,495.144153,19.936002,103.361776,31859.674915,10665.027175,624.932195,2672.490584,15417.439685,18089.930269,168638.489954,0.726923,24.231595,1069.771002,0.168059,51.829529,17.396131,1.016295,4.360754,25.229232,29.589986
3,056P98,11,1,71047.323880,67251.314181,326.068274,572.692526,2658.806644,203.172508,35.269748,128.242526,49603.843463,14525.287710,425.619288,2960.529552,3403.801341,6364.330893,152531.559789,0.931528,6.618531,390.357019,0.180436,69.797998,20.457505,0.598404,4.169929,4.795728,8.965657
4,059P93,11,1,128651.947989,112871.894252,649.543070,2109.063691,12480.372985,503.292367,37.781625,373.831223,90995.921626,16515.131891,1389.138725,4275.602559,15102.321965,19377.924524,301814.762975,0.851370,11.122295,928.074771,0.302790,70.509378,12.872487,1.094698,3.359808,11.860840,15.220647
5,114P67,11,1,56621.011818,44376.205640,634.701848,1296.679334,9934.327010,353.929881,25.168105,277.552898,32255.218415,9534.577431,482.298748,2491.568462,11579.795864,14071.364326,147814.625231,0.768995,20.560589,853.783053,0.464672,57.859000,16.488974,0.818066,4.219665,20.149624,24.369288


In [76]:
truck_stations[truck_stations.FC_Old == 11].reset_index(drop=True)

,Station,FC_Old,FC_New,TotalCount_AADT,naxle2_AADT,naxle3_AADT,naxle4_AADT,naxle5_AADT,naxle6_AADT,naxle7_AADT,motorcycles_AADT,passenger_cars_AADT,other2axle4tire_AADT,bus_AADT,singleunit_AADT,combination_AADT,Truck_Count_AADT,total_axles,axle_factor,Peakhour_truck_percentage,Peakhr_Truck_Count,motorcycle_percentage,passenger_cars_percentage,other2axle4tire_percentage,bus_percentage,singleunit_percentage,combination_percentage,truck_percentage
0,019P97,11,1,85369.045991,82719.924135,247.103381,398.118318,1847.710692,152.166274,4.023192,152.971305,67406.919025,12402.399764,496.163522,2541.624214,2368.968160,4910.592374,177953.345126,0.959454,4.608046,341.422563,0.179188,78.959438,14.527982,0.581198,2.977220,2.774973,5.752193
1,034I05,11,1,80744.056345,67875.791915,440.075449,1169.856767,10847.587019,398.759144,11.986049,299.504980,50636.169801,12911.895755,958.339448,3558.459506,12379.686855,15938.146361,198465.629554,0.813512,14.691107,931.087217,0.376376,62.722745,15.941445,1.211439,4.399176,15.348819,19.747995
2,047P68,11,1,61342.926329,45465.863612,411.984835,1389.597343,13560.400385,495.144153,19.936002,103.361776,31859.674915,10665.027175,624.932195,2672.490584,15417.439685,18089.930269,168638.489954,0.726923,24.231595,1069.771002,0.168059,51.829529,17.396131,1.016295,4.360754,25.229232,29.589986
3,056P98,11,1,71047.323880,67251.314181,326.068274,572.692526,2658.806644,203.172508,35.269748,128.242526,49603.843463,14525.287710,425.619288,2960.529552,3403.801341,6364.330893,152531.559789,0.931528,6.618531,390.357019,0.180436,69.797998,20.457505,0.598404,4.169929,4.795728,8.965657
4,059P93,11,1,128651.947989,112871.894252,649.543070,2109.063691,12480.372985,503.292367,37.781625,373.831223,90995.921626,16515.131891,1389.138725,4275.602559,15102.321965,19377.924524,301814.762975,0.851370,11.122295,928.074771,0.302790,70.509378,12.872487,1.094698,3.359808,11.860840,15.220647
5,114P67,11,1,62017.834554,48143.070818,475.458181,1780.251832,11186.028694,405.589744,27.435287,251.453297,35499.561355,10140.697497,485.241758,2273.148352,13367.732295,15640.880647,163389.252442,0.759142,21.545578,936.958791,0.405453,57.240891,16.351260,0.782423,3.665314,21.554658,25.219972
